In [20]:
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import torchaudio

In [34]:
from torch import nn
import librosa
import pandas as pd
import scipy
import matplotlib.pyplot as plt
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import torchaudio

In [5]:
data_path = "TrainData"
train_x,train_y,test_x,test_y,valid_x,valid_y = [],[],[],[],[],[]
data_rate = 80

In [6]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
# cuda(gpu) 사용 가능한 경우 gpu 사용 하고 else 인 경우 cpu 사용
print("실행여부 확인")

실행여부 확인


In [18]:
!apt-get install -y p7zip-full

Exception ignored in: <function _releaseLock at 0x7f935cc75c10>
Traceback (most recent call last):
  File "/usr/lib/python3.8/logging/__init__.py", line 227, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  p7zip
Suggested packages:
  p7zip-rar
The following NEW packages will be installed:
  p7zip p7zip-full
0 upgraded, 2 newly installed, 0 to remove and 68 not upgraded.
Need to get 1545 kB of archives.
After this operation, 5896 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 p7zip amd64 16.02+dfsg-7build1 [358 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 p7zip-full amd64 16.02+dfsg-7build1 [1187 kB]
Fetched 1545 kB in 2s (774 kB/s)    
debconf: delaying package configuration, since apt-utils is not installed
Selecting previously unselected package p7zip.
(Reading database ... 16670 files and directories currently installed.)
Preparing to unpack .../p7zip_16.02+dfsg-7build1_amd64.deb ...
Unpacking p7zip (16.02+dfsg-7build1) ...
Selecting previously unselected package 

In [19]:
!apt-get install -y p7zip-full

Reading package lists... Done
Building dependency tree       
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-7build1).
0 upgraded, 0 newly installed, 0 to remove and 68 not upgraded.


In [20]:
!7z x KsponSpeech_01.zip -o"unzipped_Speech"


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=C.UTF-8,Utf16=on,HugeFiles=on,64 bits,48 CPUs Intel(R) Xeon(R) Silver 4310 CPU @ 2.10GHz (606A6),ASM,AES-NI)

Scanning the drive for archives:
1 file, 15301122689 bytes (15 GiB)

Extracting archive: KsponSpeech_01.zip

ERRORS:
Headers Error
Unconfirmed start of archive


WARNINGS:
There are data after the end of archive

--
Path = KsponSpeech_01.zip
Type = zip
ERRORS:
Headers Error
Unconfirmed start of archive
WARNINGS:
There are data after the end of archive
Physical Size = 2147585904
Tail Size = 13153536785

ERROR: Data Error : KsponSpeech_01/KsponSpeech_0018/KsponSpeech_017485.pcm

Sub items Errors: 1

Archives with Errors: 1

Warnings: 1

Open Errors: 1

Sub items Errors: 1


In [2]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [9]:
import os
import torch
import numpy as np
import librosa
import matplotlib.pyplot as plt
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import Dataset, DatasetDict, Audio
from dataclasses import dataclass
from typing import Dict, List, Union


def read_npz(file_path, sr=16000):
    """
    NPZ 파일을 읽어 numpy 배열로 반환합니다.
    """
    # npz 파일 로드
    with np.load(file_path) as data:
        # npz 파일에 저장된 첫 번째 배열 가져오기
        # 일반적으로 npz 파일에는 'arr_0'이라는 키로 데이터가 저장됩니다
        if 'arr_0' in data:
            audio_data = data['arr_0']
        # 또는 특정 키가 있는 경우 그 키를 사용할 수 있습니다
        # 예: audio_data = data['audio']
        else:
            # 첫 번째 키 사용
            key = list(data.keys())[0]
            audio_data = data[key]

    # 이미 정규화되어 있는지 확인하고, 필요한 경우 정규화
    if audio_data.dtype in [np.int16, np.int8]:
        max_value = float(2 ** (15 if audio_data.dtype == np.int16 else 7))
        audio_data = audio_data.astype(np.float32) / max_value

    return audio_data, sr
def load_text(file_path):
    # 일반적인 한국어 인코딩 시도
    encodings = ['utf-8', 'cp949', 'euc-kr']

    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as f:
                text = f.read().strip()
            return text
        except UnicodeDecodeError:
            continue

    return "텍스트 로드 실패"

def get_file_paths_npz(base_directory):
    """
    디렉토리에서 PCM 파일과 텍스트 파일의 경로를 가져옵니다.
    """
    base = []

    for i in range(1, 19):
        if i > 10 :
            base.append(base_directory + f"/KsponSpeech_00{i}")
        else:
            base.append(base_directory + f"/KsponSpeech_000{i}")

    npz_files = []

    for base_path in base:
        for root, _, files in os.walk(base_path):
            for file in files:
                if file.endswith('.npz'):
                    npz_path = os.path.join(root, file)
                    if os.path.exists(npz_path):
                        npz_files.append(npz_path)
                        print("성공")
                    else:
                        print("실패")
        return npz_files

def get_file_paths_txt(npz_files):
    txt_files = []
    # 모든 파일을 검색
    for path in npz_files:
        temp = path.split(".")
        temp_p = temp[1] + "_g2p"
        temp_path = temp[0] + temp_p
        txt_files.append(temp_path)

    return txt_files

def create_dataset(npz_files, txt_files, max_samples=None):
    """
    PCM 파일과 텍스트 파일로부터 데이터셋을 생성합니다.
    """
    data = {"audio": [], "text": []}
    # 최대 샘플 수 제한
    if max_samples is not None:
        npz_files = npz_files[:max_samples]
        txt_files = txt_files[:max_samples]

    for npz_file, txt_file in zip(npz_files, txt_files):
        data["audio"].append(npz_file)
        data["text"].append(load_text(txt_file))

    return Dataset.from_dict(data)

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: WhisperProcessor

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        batch = self.processor.feature_extractor.pad(
            {"input_features": [feature["input_features"] for feature in features]},
            return_tensors="pt"
        )

        labels_batch = self.processor.tokenizer.pad(
            {"input_ids": [feature["labels"] for feature in features]},
            return_tensors="pt"
        )

        labels = labels_batch["input_ids"].masked_fill(
            labels_batch["input_ids"] == self.processor.tokenizer.pad_token_id,
            -100
        )

        batch["labels"] = labels

        return batch

def train_whisper_model():
    base_directory = "PreprocessData/KsponSpeech_01"
    output_dir = "whisper_finetuned"
    pcm_files = get_file_paths_npz(base_directory)
    txt_files = get_file_paths_txt(pcm_files)
    print(f"총 {len(pcm_files)}개의 파일이 로드되었습니다.")

    dataset = create_dataset(pcm_files, txt_files)

    def map_to_array(batch):
        arrays = []
        rates = []

        for audio_path in batch["audio"]:
            audio_array, sampling_rate = read_npz(audio_path)
            arrays.append(audio_array)
            rates.append(sampling_rate)

        batch["audio"] = [{"array": arr, "sampling_rate": sr} for arr, sr in zip(arrays, rates)]
        return batch

    # 데이터셋에 Audio 형식 적용 (배치 처리)
    dataset = dataset.map(map_to_array, batched=True, batch_size=8)

    # 훈련/검증/테스트 세트 분할
    train_test_valid = dataset.train_test_split(test_size=0.2)
    test_valid = train_test_valid["test"].train_test_split(test_size=0.5)

    datasets = DatasetDict({
        "train": train_test_valid["train"],
        "test": test_valid["test"],
        "validation": test_valid["train"]
    })

    # Whisper 모델 및 프로세서 로드
    model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
    processor = WhisperProcessor.from_pretrained("openai/whisper-small")

    def prepare_dataset(batch):
        # 오디오 처리
        audio = batch["audio"]

        # 특성 추출
        input_features = processor.feature_extractor(
            audio["array"],
            sampling_rate=audio["sampling_rate"]
        ).input_features[0]

        # NumPy 배열을 PyTorch 텐서로 변환 후 데이터 타입 조정
        if isinstance(input_features, np.ndarray):
            input_features = torch.from_numpy(input_features)

        # 이제 텐서가 되었으므로 데이터 타입 변환 가능
        if hasattr(model, "dtype"):
            input_features = input_features.to(model.dtype)

        # 텍스트 처리
        labels = processor.tokenizer(batch["text"]).input_ids

        return {
            "input_features": input_features,
            "labels": labels
        }

    # 데이터셋 전처리 적용 (비배치 함수)
    processed_datasets = DatasetDict({
        split: dataset.map(
            prepare_dataset,
            remove_columns=dataset.column_names
        )
        for split, dataset in datasets.items()
    })

    # 데이터 콜레이터 설정
    data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

    def compute_metrics(pred):
        pred_ids = pred.predictions
        label_ids = pred.label_ids

        # pred_ids가 튜플인 경우 (일반적으로 첫 번째 요소가 예측값)
        if isinstance(pred_ids, tuple):
            pred_ids = pred_ids[0]

        # 이제 pred_ids가 텐서/배열인지 확인하고 3차원인 경우 처리
        if hasattr(pred_ids, 'shape') and len(pred_ids.shape) > 2:
            pred_ids = np.argmax(pred_ids, axis=-1)

        # 패딩 토큰 무시
        label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

        # 예측과 레이블을 텍스트로 디코딩
        pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
        label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

        # WER 계산 
        # 참고: jiwer 라이브러리가 임포트되었는지 확인
        import jiwer  # 이 라인 추가
        wer = jiwer.wer(label_str, pred_str)

        return {"wer": wer}

    # 훈련 인자 설정
    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=2,
        learning_rate=1e-5,
        warmup_steps=500,
        max_steps=4000,
        gradient_checkpointing=True,
        # fp16=True,
        fp16_full_eval=False,
        evaluation_strategy="steps",  # 올바른 파라미터명 사용
        eval_steps=500,
        save_steps=500,
        logging_steps=25,
        report_to=["tensorboard"],
        load_best_model_at_end=True,
        metric_for_best_model="wer",
        greater_is_better=False,
        push_to_hub=False,
    )

    # 훈련기 설정
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=processed_datasets["train"],
        eval_dataset=processed_datasets["validation"],
        data_collator=data_collator,
        tokenizer=processor.tokenizer,  # processing_class 대신 tokenizer 사용
        compute_metrics=compute_metrics,
    )

    # 모델 훈련
    trainer.train()

    # 모델 저장
    trainer.save_model(output_dir)
    processor.save_pretrained(output_dir)

    # 테스트 평가
    test_results = trainer.evaluate(processed_datasets["test"])
    print(f"테스트 결과: {test_results}")

    return model, processor


# 모델 추론 함수
def transcribe_audio(model, processor, audio_file):
    """
    훈련된 모델을 사용하여 오디오 파일을 텍스트로 변환합니다.
    """
    # PCM 파일 로드
    audio_array, sr = read_npz(audio_file)

    # 특성 추출
    input_features = processor.feature_extractor(
        audio_array,
        sampling_rate=sr,
        return_tensors="pt"
    ).input_features

    # 모델을 통한 예측
    with torch.no_grad():
        predicted_ids = model.generate(input_features)

    # 예측된 토큰을 텍스트로 디코딩
    transcription = processor.tokenizer.batch_decode(
        predicted_ids,
        skip_special_tokens=True
    )[0]

    return transcription

In [10]:
if __name__ == "__main__":
    # 모델 훈련
    model, processor = train_whisper_model()
    
    # 테스트할 디렉터리 설정
    test_directory = "PreprocessData/KsponSpeech_01"
    
    # 테스트 파일 가져오기
    pcm_files, txt_files = get_file_paths(test_directory)
    print(f"테스트할 파일: {len(pcm_files)}개")
    
    # 모든 파일 테스트 (또는 원하는 만큼 제한)
    for i in range(len(pcm_files)):
        test_audio = pcm_files[i]
        reference_text = load_text(txt_files[i])
        
        transcription = transcribe_audio(model, processor, test_audio)
        
        print(f"파일: {os.path.basename(test_audio)}")
        print(f"원본 텍스트: {reference_text}")
        print(f"변환 결과: {transcription}")
        print("-" * 50)

총 0개의 파일이 로드되었습니다.


/usr/local/lib/python3.8/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_628/2787886146.py:252: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
max_steps is given, it will override any value given in num_train_epochs


ValueError: No columns in the dataset match the model's forward method signature. The following columns have been ignored: []. Please check the dataset and model. You may need to set `remove_unused_columns=False` in `TrainingArguments`.

In [8]:
# 현재 작업 디렉토리 확인
import os
print("현재 작업 디렉토리:", os.getcwd())

# 특정 디렉토리의 구조 확인
def print_directory_structure(directory, level=0, max_depth=3, max_files=5):
    if level > max_depth:
        print("  " * level + "...")
        return
    
    try:
        items = os.listdir(directory)
        dirs = []
        files = []
        
        for item in items:
            item_path = os.path.join(directory, item)
            if os.path.isdir(item_path):
                dirs.append(item)
            else:
                files.append(item)
        
        for d in sorted(dirs):
            print("  " * level + "📁 " + d)
            print_directory_structure(os.path.join(directory, d), level + 1, max_depth, max_files)
        
        if files:
            print("  " * level + f"📄 파일 {len(files)}개:")
            for i, f in enumerate(sorted(files)):
                if i >= max_files:
                    print("  " * (level + 1) + f"... 외 {len(files) - max_files}개")
                    break
                print("  " * (level + 1) + f)
    except PermissionError:
        print("  " * level + "❌ 접근 권한 없음")
    except Exception as e:
        print("  " * level + f"❌ 오류: {e}")

# 확인하고 싶은 디렉토리 경로 (원하는 경로로 수정)
# 예: data_directory = "unzipped_Speech"
# 또는 base_directory에 지정된 경로를 사용
data_directory = "PreprocessData"
print(f"\n{data_directory} 디렉토리 구조:")
print_directory_structure(data_directory)

# KsponSpeech_01 디렉토리가 있는지 구체적으로 확인
target_dir = "PreprocessData_Speech/KsponSpeech_01"
if os.path.exists(target_dir):
    print(f"\n{target_dir}가 존재합니다.")
    print(f"{target_dir} 디렉토리 구조:")
    print_directory_structure(target_dir)
else:
    print(f"\n❌ {target_dir}가 존재하지 않습니다.")

현재 작업 디렉토리: /data

PreprocessData 디렉토리 구조:
📁 .ipynb_checkpoints
📁 KsponSpeech_01
  📁 .ipynb_checkpoints
  📁 KsponSpeech_0001
    📄 파일 997개:
      KsponSpeech_000001_combined_features.npy
      KsponSpeech_000002_combined_features.npy
      KsponSpeech_000003_combined_features.npy
      KsponSpeech_000004_combined_features.npy
      KsponSpeech_000005_combined_features.npy
      ... 외 992개
  📁 KsponSpeech_0001_g2p
    📁 .ipynb_checkpoints
      📄 파일 10개:
        KsponSpeech_000001-checkpoint.txt
        KsponSpeech_000002-checkpoint.txt
        KsponSpeech_000003-checkpoint.txt
        KsponSpeech_000004-checkpoint.txt
        KsponSpeech_000009-checkpoint.txt
        ... 외 5개
    📄 파일 1000개:
      KsponSpeech_000001.txt
      KsponSpeech_000002.txt
      KsponSpeech_000003.txt
      KsponSpeech_000004.txt
      KsponSpeech_000005.txt
      ... 외 995개
  📁 KsponSpeech_0002
    📄 파일 160개:
      KsponSpeech_001001_combined_features.npy
      KsponSpeech_001003_combined_features.npy
      K

In [28]:
model_path = "whisper_finetuned/saved_whisper_model.pt"
torch.save(model.state_dict(), model_path)

In [23]:
!pip install transformers[torch]

In [1]:
import os
import torch
import numpy as np
import librosa
import matplotlib.pyplot as plt
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import Dataset, DatasetDict, Audio
from dataclasses import dataclass
from typing import Dict, List, Union

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
def load_text(file_path):
    # 일반적인 한국어 인코딩 시도
    encodings = ['utf-8', 'cp949', 'euc-kr']

    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as f:
                text = f.read().strip()
            return text
        except UnicodeDecodeError:
            continue

    return "텍스트 로드 실패"


def get_file_paths_npy(base_directory):
    """
    디렉토리에서 NP 파일(combined_features.npy)의 경로를 가져옵니다.
    """
    npy_files = []

    # 디렉토리 구조에 맞게 폴더 목록 생성
    base_folders = []
    for i in range(1, 19):
        if i > 9:
            base_folders.append(f"KsponSpeech_00{i}")
        else:
            base_folders.append(f"KsponSpeech_000{i}")

    # 모든 폴더 순회
    for folder in base_folders:
        folder_path = os.path.join(base_directory, folder)
        
        # 폴더가 존재하는지 확인
        if os.path.isdir(folder_path):
            print(f"폴더 검색 중: {folder_path}")
            
            # 폴더 내의 파일 검색
            for root, _, files in os.walk(folder_path):
                for file in files:
                    if file.endswith('_combined_features.npy'):
                        npy_path = os.path.join(root, file)
                        npy_files.append(npy_path)
                        if len(npy_files) % 100 == 0:  # 100개마다 로그 출력
                            print(f"찾은 파일 수: {len(npy_files)}")
        else:
            print(f"폴더가 존재하지 않음: {folder_path}")
    
    print(f"총 {len(npy_files)}개의 NP 파일을 찾았습니다.")
    return npy_files


def get_file_paths_txt(npy_files):
    """
    NP 파일 경로로부터 해당하는 텍스트 파일 경로를 생성합니다.
    """
    txt_files = []
    
    for npy_path in npy_files:
        # 파일명 추출 (경로와 확장자 제거)
        file_name = os.path.basename(npy_path).replace('_combined_features.npy', '')
        
        # 디렉토리 구조 분석
        dir_path = os.path.dirname(npy_path)
        parent_dir = os.path.basename(dir_path)
        
        # 해당 G2P 폴더 경로 생성
        g2p_dir = parent_dir + "_g2p"
        g2p_dir_path = os.path.join(os.path.dirname(dir_path), g2p_dir)
        
        # 텍스트 파일 경로 생성
        txt_path = os.path.join(g2p_dir_path, file_name + ".txt")
        
        # 파일이 존재하는지 확인 (선택적)
        if os.path.exists(txt_path):
            txt_files.append(txt_path)
        else:
            print(f"경고: 텍스트 파일이 존재하지 않습니다: {txt_path}")
    
    print(f"총 {len(txt_files)}개의 텍스트 파일을 매핑했습니다.")
    return txt_files


def create_dataset(npy_files, txt_files, max_samples=None):
    """
    NP 파일과 텍스트 파일로부터 데이터셋을 생성합니다.
    """
    data = {"audio": [], "text": []}
    # 최대 샘플 수 제한
    if max_samples is not None:
        npy_files = npy_files[:max_samples]
        txt_files = txt_files[:max_samples]

    for npy_file, txt_file in zip(npy_files, txt_files):
        data["audio"].append(npy_file)
        data["text"].append(load_text(txt_file))

    return Dataset.from_dict(data)


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: WhisperProcessor

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        batch = self.processor.feature_extractor.pad(
            {"input_features": [feature["input_features"] for feature in features]},
            return_tensors="pt"
        )

        labels_batch = self.processor.tokenizer.pad(
            {"input_ids": [feature["labels"] for feature in features]},
            return_tensors="pt"
        )

        labels = labels_batch["input_ids"].masked_fill(
            labels_batch["input_ids"] == self.processor.tokenizer.pad_token_id,
            -100
        )

        batch["labels"] = labels

        return batch


def train_whisper_model():
    # 경로 수정
    base_directory = "PreprocessData/KsponSpeech_01"  # unzipped_Speech에서 PreprocessData로 변경
    output_dir = "whisper_finetuned"
    
    # 파일 경로 가져오기 함수 이름 변경
    npy_files = get_file_paths_npy(base_directory)
    
    if len(npy_files) == 0:
        print("파일을 찾을 수 없습니다. 경로와 파일 패턴을 확인하세요.")
        return None, None
    
    txt_files = get_file_paths_txt(npy_files)
    print(f"총 {len(npy_files)}개의 오디오 파일과 {len(txt_files)}개의 텍스트 파일이 로드되었습니다.")
    
    # 파일 수가 일치하지 않으면 경고
    if len(npy_files) != len(txt_files):
        print(f"경고: 오디오 파일 수({len(npy_files)})와 텍스트 파일 수({len(txt_files)})가 일치하지 않습니다.")
        # 일치하는 파일만 사용
        if len(npy_files) > len(txt_files):
            npy_files = npy_files[:len(txt_files)]
        else:
            txt_files = txt_files[:len(npy_files)]
    
    dataset = create_dataset(npy_files, txt_files)

    def map_to_array(batch):
        arrays = []
        rates = []

        for audio_path in batch["audio"]:
            # npy 파일 로드
            audio_array = np.load(audio_path)
            # float32로 변환하고 정규화 (필요한 경우)
            if audio_array.dtype in [np.int16, np.int8]:
                max_value = float(2 ** (15 if audio_array.dtype == np.int16 else 7))
                audio_array = audio_array.astype(np.float32) / max_value
            elif audio_array.dtype != np.float32:
                audio_array = audio_array.astype(np.float32)
            
            # 샘플링 레이트는 고정 (KsponSpeech는 16kHz)
            sampling_rate = 16000
            
            arrays.append(audio_array)
            rates.append(sampling_rate)

        batch["audio"] = [{"array": arr, "sampling_rate": sr} for arr, sr in zip(arrays, rates)]
        return batch

    # 데이터셋에 Audio 형식 적용 (배치 처리)
    dataset = dataset.map(map_to_array, batched=True, batch_size=8)

    # 훈련/검증/테스트 세트 분할
    train_test_valid = dataset.train_test_split(test_size=0.2)
    test_valid = train_test_valid["test"].train_test_split(test_size=0.5)

    datasets = DatasetDict({
        "train": train_test_valid["train"],
        "test": test_valid["test"],
        "validation": test_valid["train"]
    })

    # Whisper 모델 및 프로세서 로드
    model = WhisperForConditionalGeneration.from_pretrained(
    "openai/whisper-small", 
    use_cache=False,
    low_cpu_mem_usage=True
    # torch_dtype=torch.float16 제거
    )
    processor = WhisperProcessor.from_pretrained("openai/whisper-small")

    def prepare_dataset(batch):
        # 오디오 처리
        audio = batch["audio"]
    
        # 특성 추출
        input_features = processor.feature_extractor(
            audio["array"],
            sampling_rate=audio["sampling_rate"]
        ).input_features[0]
    
        # NumPy 배열을 PyTorch 텐서로 변환
        if isinstance(input_features, np.ndarray):
            input_features = torch.from_numpy(input_features)
    
        # 텐서 타입 변환 (중요: 타입을 명시적으로 float16으로 설정하지 않음)
        # input_features = input_features.to(model.dtype)  # 이 줄을 제거하거나 아래와 같이 수정
    
        # 텍스트 처리
        labels = processor.tokenizer(batch["text"]).input_ids
    
        return {
            "input_features": input_features,
            "labels": labels
        }

    # 데이터셋 전처리 적용 (비배치 함수)
    processed_datasets = DatasetDict({
        split: dataset.map(
            prepare_dataset,
            remove_columns=dataset.column_names
        )
        for split, dataset in datasets.items()
    })

    # 데이터 콜레이터 설정
    data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

    def compute_metrics(pred):
        pred_ids = pred.predictions
        label_ids = pred.label_ids

        # pred_ids가 튜플인 경우 (일반적으로 첫 번째 요소가 예측값)
        if isinstance(pred_ids, tuple):
            pred_ids = pred_ids[0]

        # 이제 pred_ids가 텐서/배열인지 확인하고 3차원인 경우 처리
        if hasattr(pred_ids, 'shape') and len(pred_ids.shape) > 2:
            pred_ids = np.argmax(pred_ids, axis=-1)

        # 패딩 토큰 무시
        label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

        # 예측과 레이블을 텍스트로 디코딩
        pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
        label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

        # WER 계산 
        # 참고: jiwer 라이브러리가 임포트되었는지 확인
        import jiwer  # 이 라인 추가
        wer = jiwer.wer(label_str, pred_str)

        return {"wer": wer}

    # 훈련 인자 설정
    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=2,
        learning_rate=1e-5,
        warmup_steps=500,
        max_steps=4000,
        gradient_checkpointing=True,
        fp16=True,  # 혼합 정밀도는 여전히 활성화
        # fp16_full_eval=True 제거 (평가를 하지 않으므로 불필요)
        evaluation_strategy="no",
        save_steps=500,
        logging_steps=25,
        report_to=["tensorboard"],
        push_to_hub=False,
        # 필요하지 않은 옵션들 제거
    )

    # 훈련기 설정
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=processed_datasets["train"],
        # eval_dataset=processed_datasets["validation"],  # 이 줄 제거 또는 주석 처리
        data_collator=data_collator,
        tokenizer=processor.tokenizer,
        # compute_metrics=compute_metrics,  # 이 줄 제거 또는 주석 처리
    )

    # 모델 훈련
    trainer.train()

    # 모델 저장
    trainer.save_model(output_dir)
    processor.save_pretrained(output_dir)

    # 테스트 평가
    test_results = trainer.evaluate(processed_datasets["test"])
    print(f"테스트 결과: {test_results}")

    return model, processor


# 모델 추론 함수
def transcribe_audio(model, processor, audio_file):
    """
    훈련된 모델을 사용하여 오디오 파일을 텍스트로 변환합니다.
    """
    # NP 파일 로드
    audio_array = np.load(audio_file)
    
    # float32로 변환하고 정규화 (필요한 경우)
    if audio_array.dtype in [np.int16, np.int8]:
        max_value = float(2 ** (15 if audio_array.dtype == np.int16 else 7))
        audio_array = audio_array.astype(np.float32) / max_value
    elif audio_array.dtype != np.float32:
        audio_array = audio_array.astype(np.float32)
    
    # 샘플링 레이트는 고정 (KsponSpeech는 16kHz)
    sr = 16000

    # 특성 추출
    input_features = processor.feature_extractor(
        audio_array,
        sampling_rate=sr,
        return_tensors="pt"
    ).input_features

    # 모델을 통한 예측
    with torch.no_grad():
        predicted_ids = model.generate(input_features)

    # 예측된 토큰을 텍스트로 디코딩
    transcription = processor.tokenizer.batch_decode(
        predicted_ids,
        skip_special_tokens=True
    )[0]

    return transcription


if __name__ == "__main__":
    # 모델 훈련
    model, processor = train_whisper_model()
    
    if model is None or processor is None:
        print("모델 훈련에 실패했습니다.")
        exit()
    
    # 테스트할 디렉터리 설정
    test_directory = "PreprocessData/KsponSpeech_01"
    
    # 테스트 파일 가져오기 - 함수 이름 변경
    npy_files = get_file_paths_npy(test_directory)
    txt_files = get_file_paths_txt(npy_files)
    
    print(f"테스트할 파일: {len(npy_files)}개")
    
    # 테스트할 파일 수 제한 (선택적)
    max_test_files = 10
    test_npy_files = npy_files[:max_test_files]
    test_txt_files = txt_files[:max_test_files]
    
    # 모든 파일 테스트
    for i in range(len(test_npy_files)):
        test_audio = test_npy_files[i]
        reference_text = load_text(test_txt_files[i])
        
        transcription = transcribe_audio(model, processor, test_audio)
        
        print(f"파일: {os.path.basename(test_audio)}")
        print(f"원본 텍스트: {reference_text}")
        print(f"변환 결과: {transcription}")
        print("-" * 50)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-25 01:10:28.596516: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-25 01:10:28.645096: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-25 01:10:29.371019: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0001
찾은 파일 수: 100
찾은 파일 수: 200
찾은 파일 수: 300
찾은 파일 수: 400
찾은 파일 수: 500
찾은 파일 수: 600
찾은 파일 수: 700
찾은 파일 수: 800
찾은 파일 수: 900
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0002
찾은 파일 수: 1000
찾은 파일 수: 1100
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0003
찾은 파일 수: 1200
찾은 파일 수: 1300
찾은 파일 수: 1400
찾은 파일 수: 1500
찾은 파일 수: 1600
찾은 파일 수: 1700
찾은 파일 수: 1800
찾은 파일 수: 1900
찾은 파일 수: 2000
찾은 파일 수: 2100
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0004
찾은 파일 수: 2200
찾은 파일 수: 2300
찾은 파일 수: 2400
찾은 파일 수: 2500
찾은 파일 수: 2600
찾은 파일 수: 2700
찾은 파일 수: 2800
찾은 파일 수: 2900
찾은 파일 수: 3000
찾은 파일 수: 3100
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0005
찾은 파일 수: 3200
찾은 파일 수: 3300
찾은 파일 수: 3400
찾은 파일 수: 3500
찾은 파일 수: 3600
찾은 파일 수: 3700
찾은 파일 수: 3800
찾은 파일 수: 3900
찾은 파일 수: 4000
찾은 파일 수: 4100
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0006
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0007
찾은 파일 수: 4200
찾은 파일 수: 4300
찾은 파일 수: 4400
찾

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1271/1271 [04:24<00:00,  4.80 examples/s]
/usr/local/lib/python3.8/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_18222/1156997083.py:292: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.8/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
25,10.502600
50,8.572000
75,7.034300
100,6.161400
125,5.573500
150,5.127000
175,4.910000
200,4.763700
225,4.537200
250,4.394500


/usr/local/lib/python3.8/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is

테스트 결과: {'eval_loss': 3.0094687938690186, 'eval_runtime': 165.0981, 'eval_samples_per_second': 7.705, 'eval_steps_per_second': 7.705, 'epoch': 6.289308176100629}
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0001
찾은 파일 수: 100
찾은 파일 수: 200
찾은 파일 수: 300
찾은 파일 수: 400
찾은 파일 수: 500
찾은 파일 수: 600
찾은 파일 수: 700
찾은 파일 수: 800
찾은 파일 수: 900
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0002
찾은 파일 수: 1000
찾은 파일 수: 1100
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0003
찾은 파일 수: 1200
찾은 파일 수: 1300
찾은 파일 수: 1400
찾은 파일 수: 1500
찾은 파일 수: 1600
찾은 파일 수: 1700
찾은 파일 수: 1800
찾은 파일 수: 1900
찾은 파일 수: 2000
찾은 파일 수: 2100
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0004
찾은 파일 수: 2200
찾은 파일 수: 2300
찾은 파일 수: 2400
찾은 파일 수: 2500
찾은 파일 수: 2600
찾은 파일 수: 2700
찾은 파일 수: 2800
찾은 파일 수: 2900
찾은 파일 수: 3000
찾은 파일 수: 3100
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0005
찾은 파일 수: 3200
찾은 파일 수: 3300
찾은 파일 수: 3400
찾은 파일 수: 3500
찾은 파일 수: 3600
찾은 파일 수: 3700
찾은 파일 수: 3800
찾은 파일 수: 3900
찾은 파일 수: 4000
찾은 파일 수

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


RuntimeError: Input type (float) and bias type (c10::Half) should be the same

In [ ]:
import os
import torch
import numpy as np
import librosa
import matplotlib.pyplot as plt
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import Dataset, DatasetDict, Audio
from dataclasses import dataclass
from typing import Dict, List, Union

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

def load_text(file_path):
    # 일반적인 한국어 인코딩 시도
    encodings = ['utf-8', 'cp949', 'euc-kr']

    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as f:
                text = f.read().strip()
            return text
        except UnicodeDecodeError:
            continue

    return "텍스트 로드 실패"


def get_file_paths_npy(base_directory):
    """
    디렉토리에서 NP 파일(combined_features.npy)의 경로를 가져옵니다.
    """
    npy_files = []

    # 디렉토리 구조에 맞게 폴더 목록 생성
    base_folders = []
    for i in range(1, 19):
        if i > 9:
            base_folders.append(f"KsponSpeech_00{i}")
        else:
            base_folders.append(f"KsponSpeech_000{i}")

    # 모든 폴더 순회
    for folder in base_folders:
        folder_path = os.path.join(base_directory, folder)
        
        # 폴더가 존재하는지 확인
        if os.path.isdir(folder_path):
            print(f"폴더 검색 중: {folder_path}")
            
            # 폴더 내의 파일 검색
            for root, _, files in os.walk(folder_path):
                for file in files:
                    if file.endswith('_combined_features.npy'):
                        npy_path = os.path.join(root, file)
                        npy_files.append(npy_path)
                        if len(npy_files) % 100 == 0:  # 100개마다 로그 출력
                            print(f"찾은 파일 수: {len(npy_files)}")
        else:
            print(f"폴더가 존재하지 않음: {folder_path}")
    
    print(f"총 {len(npy_files)}개의 NP 파일을 찾았습니다.")
    return npy_files


def get_file_paths_txt(npy_files):
    """
    NP 파일 경로로부터 해당하는 텍스트 파일 경로를 생성합니다.
    """
    txt_files = []
    
    for npy_path in npy_files:
        # 파일명 추출 (경로와 확장자 제거)
        file_name = os.path.basename(npy_path).replace('_combined_features.npy', '')
        
        # 디렉토리 구조 분석
        dir_path = os.path.dirname(npy_path)
        parent_dir = os.path.basename(dir_path)
        
        # 해당 G2P 폴더 경로 생성
        g2p_dir = parent_dir + "_g2p"
        g2p_dir_path = os.path.join(os.path.dirname(dir_path), g2p_dir)
        
        # 텍스트 파일 경로 생성
        txt_path = os.path.join(g2p_dir_path, file_name + ".txt")
        
        # 파일이 존재하는지 확인 (선택적)
        if os.path.exists(txt_path):
            txt_files.append(txt_path)
        else:
            print(f"경고: 텍스트 파일이 존재하지 않습니다: {txt_path}")
    
    print(f"총 {len(txt_files)}개의 텍스트 파일을 매핑했습니다.")
    return txt_files


def create_dataset(npy_files, txt_files, max_samples=None):
    """
    NP 파일과 텍스트 파일로부터 데이터셋을 생성합니다.
    """
    data = {"audio": [], "text": []}
    
    # 최대 샘플 수 제한
    if max_samples is not None:
        npy_files = npy_files[:max_samples]
        txt_files = txt_files[:max_samples]

    for npy_file, txt_file in zip(npy_files, txt_files):
        try:
            # 파일 존재 확인
            if os.path.exists(npy_file) and os.path.exists(txt_file):
                # npy 파일 로드 테스트
                test_array = np.load(npy_file)
                if len(test_array) > 0:  # 빈 배열 체크
                    data["audio"].append(npy_file)
                    data["text"].append(load_text(txt_file))
                else:
                    print(f"빈 오디오 파일 건너뜀: {npy_file}")
            else:
                print(f"파일 없음: {npy_file} 또는 {txt_file}")
        except Exception as e:
            print(f"파일 로드 오류: {npy_file}, 오류: {e}")
            continue

    return Dataset.from_dict(data)


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: WhisperProcessor

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        batch = self.processor.feature_extractor.pad(
            {"input_features": [feature["input_features"] for feature in features]},
            return_tensors="pt"
        )

        labels_batch = self.processor.tokenizer.pad(
            {"input_ids": [feature["labels"] for feature in features]},
            return_tensors="pt"
        )

        labels = labels_batch["input_ids"].masked_fill(
            labels_batch["input_ids"] == self.processor.tokenizer.pad_token_id,
            -100
        )

        batch["labels"] = labels

        return batch


def train_whisper_model():
    # 경로 수정
    base_directory = "PreprocessData/KsponSpeech_01"
    output_dir = "whisper_finetuned"
    
    # 파일 경로 가져오기
    npy_files = get_file_paths_npy(base_directory)
    
    if len(npy_files) == 0:
        print("파일을 찾을 수 없습니다. 경로와 파일 패턴을 확인하세요.")
        return None, None
    
    txt_files = get_file_paths_txt(npy_files)
    print(f"총 {len(npy_files)}개의 오디오 파일과 {len(txt_files)}개의 텍스트 파일이 로드되었습니다.")
    
    # 파일 수가 일치하지 않으면 경고
    if len(npy_files) != len(txt_files):
        print(f"경고: 오디오 파일 수({len(npy_files)})와 텍스트 파일 수({len(txt_files)})가 일치하지 않습니다.")
        # 일치하는 파일만 사용
        if len(npy_files) > len(txt_files):
            npy_files = npy_files[:len(txt_files)]
        else:
            txt_files = txt_files[:len(npy_files)]
    
    dataset = create_dataset(npy_files, txt_files)

    def map_to_array(batch):
        arrays = []
        rates = []

        for audio_path in batch["audio"]:
            # npy 파일 로드
            audio_array = np.load(audio_path)
            # float32로 변환하고 정규화 (필요한 경우)
            if audio_array.dtype in [np.int16, np.int8]:
                max_value = float(2 ** (15 if audio_array.dtype == np.int16 else 7))
                audio_array = audio_array.astype(np.float32) / max_value
            elif audio_array.dtype != np.float32:
                audio_array = audio_array.astype(np.float32)
            
            # 샘플링 레이트는 고정 (KsponSpeech는 16kHz)
            sampling_rate = 16000
            
            arrays.append(audio_array)
            rates.append(sampling_rate)

        batch["audio"] = [{"array": arr, "sampling_rate": sr} for arr, sr in zip(arrays, rates)]
        return batch

    # 데이터셋에 Audio 형식 적용 (배치 처리)
    dataset = dataset.map(map_to_array, batched=True, batch_size=8)

    # 훈련/검증/테스트 세트 분할
    train_test_valid = dataset.train_test_split(test_size=0.2)
    test_valid = train_test_valid["test"].train_test_split(test_size=0.5)

    datasets = DatasetDict({
        "train": train_test_valid["train"],
        "test": test_valid["test"],
        "validation": test_valid["train"]
    })

    # Whisper 모델 및 프로세서 로드 (fp16 사용)
    model = WhisperForConditionalGeneration.from_pretrained(
        "openai/whisper-small", 
        use_cache=False,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16  # fp16 사용
    )
    
    processor = WhisperProcessor.from_pretrained("openai/whisper-small")

    def prepare_dataset(batch):
        # 오디오 처리
        audio = batch["audio"]
    
        # 특성 추출
        input_features = processor.feature_extractor(
            audio["array"],
            sampling_rate=audio["sampling_rate"]
        ).input_features[0]
    
        # NumPy 배열을 PyTorch 텐서로 변환
        if isinstance(input_features, np.ndarray):
            input_features = torch.from_numpy(input_features)
    
        # 타입을 모델과 일치시킴 (fp16)
        input_features = input_features.to(model.dtype)
    
        # 텍스트 처리
        labels = processor.tokenizer(batch["text"]).input_ids
    
        return {
            "input_features": input_features,
            "labels": labels
        }

    # 데이터셋 전처리 적용
    processed_datasets = DatasetDict({
        split: dataset.map(
            prepare_dataset,
            remove_columns=dataset.column_names
        )
        for split, dataset in datasets.items()
    })

    # 데이터 콜레이터 설정
    data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

    def compute_metrics(pred):
        pred_ids = pred.predictions
        label_ids = pred.label_ids

        # pred_ids가 튜플인 경우 (일반적으로 첫 번째 요소가 예측값)
        if isinstance(pred_ids, tuple):
            pred_ids = pred_ids[0]

        # 이제 pred_ids가 텐서/배열인지 확인하고 3차원인 경우 처리
        if hasattr(pred_ids, 'shape') and len(pred_ids.shape) > 2:
            pred_ids = np.argmax(pred_ids, axis=-1)

        # 패딩 토큰 무시
        label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

        # 예측과 레이블을 텍스트로 디코딩
        pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
        label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

        # WER 계산
        try:
            import jiwer
            wer = jiwer.wer(label_str, pred_str)
        except ImportError:
            print("jiwer 라이브러리가 설치되지 않았습니다. WER 계산을 건너뜁니다.")
            wer = 0.0

        return {"wer": wer}

    # 훈련 인자 설정 (안정적인 fp16 설정)
    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=2,
        learning_rate=5e-6,  # 학습률 낮춤 (원래 1e-5에서)
        warmup_steps=500,
        max_steps=4000,
        gradient_checkpointing=True,
        fp16=True,
        max_grad_norm=1.0,  # gradient clipping 적당히 설정
        dataloader_pin_memory=False,
        evaluation_strategy="no",
        save_steps=500,
        logging_steps=25,
        report_to=["tensorboard"],
        push_to_hub=False,
    )

    # 훈련기 설정
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=processed_datasets["train"],
        data_collator=data_collator,
        tokenizer=processor.tokenizer,
    )

    # 모델 훈련
    print("모델 훈련을 시작합니다...")
    trainer.train()

    # 모델 저장
    print("모델을 저장합니다...")
    trainer.save_model(output_dir)
    processor.save_pretrained(output_dir)

    # 테스트 평가
    print("테스트 평가를 수행합니다...")
    try:
        test_results = trainer.evaluate(processed_datasets["test"])
        print(f"테스트 결과: {test_results}")
    except Exception as e:
        print(f"테스트 평가 중 오류 발생: {e}")

    return model, processor


def transcribe_audio(model, processor, audio_file):
    """
    훈련된 모델을 사용하여 오디오 파일을 텍스트로 변환합니다.
    """
    try:
        # NP 파일 로드
        audio_array = np.load(audio_file)
        
        # float32로 변환하고 정규화 (필요한 경우)
        if audio_array.dtype in [np.int16, np.int8]:
            max_value = float(2 ** (15 if audio_array.dtype == np.int16 else 7))
            audio_array = audio_array.astype(np.float32) / max_value
        elif audio_array.dtype != np.float32:
            audio_array = audio_array.astype(np.float32)
        
        # 샘플링 레이트는 고정 (KsponSpeech는 16kHz)
        sr = 16000

        # 특성 추출
        input_features = processor.feature_extractor(
            audio_array,
            sampling_rate=sr,
            return_tensors="pt"
        ).input_features
        
        # 타입을 모델과 일치시킴
        input_features = input_features.to(model.dtype)
        
        # GPU 사용하는 경우
        if torch.cuda.is_available():
            input_features = input_features.cuda()
            model = model.cuda()

        # 모델을 통한 예측
        with torch.no_grad():
            predicted_ids = model.generate(
                input_features,
                language="korean",  # 한국어 명시
                task="transcribe"
            )

        # 예측된 토큰을 텍스트로 디코딩
        transcription = processor.tokenizer.batch_decode(
            predicted_ids,
            skip_special_tokens=True
        )[0]

        return transcription
        
    except Exception as e:
        print(f"음성 인식 오류: {e}")
        return "음성 인식 실패"


if __name__ == "__main__":
    print("Whisper 한국어 파인튜닝을 시작합니다...")
    
    # 모델 훈련
    model, processor = train_whisper_model()
    
    if model is None or processor is None:
        print("모델 훈련에 실패했습니다.")
        exit()
    
    print("모델 훈련이 완료되었습니다!")
    
    # 테스트할 디렉터리 설정
    test_directory = "PreprocessData/KsponSpeech_01"
    
    # 테스트 파일 가져오기
    npy_files = get_file_paths_npy(test_directory)
    txt_files = get_file_paths_txt(npy_files)
    
    print(f"테스트할 파일: {len(npy_files)}개")
    
    # 테스트할 파일 수 제한 (선택적)
    max_test_files = 10
    test_npy_files = npy_files[:max_test_files]
    test_txt_files = txt_files[:max_test_files]
    
    print("테스트를 시작합니다...")
    
    # 모든 파일 테스트
    for i in range(len(test_npy_files)):
        test_audio = test_npy_files[i]
        reference_text = load_text(test_txt_files[i])
        
        transcription = transcribe_audio(model, processor, test_audio)
        
        print(f"파일: {os.path.basename(test_audio)}")
        print(f"원본 텍스트: {reference_text}")
        print(f"변환 결과: {transcription}")
        print("-" * 50)
    
    print("모든 작업이 완료되었습니다!")

In [1]:
import os
import torch
import numpy as np
import librosa
import matplotlib.pyplot as plt
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import Dataset, DatasetDict, Audio
from dataclasses import dataclass
from typing import Dict, List, Union
# 오전 12시 6분 밤에 돌린거
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

def load_text(file_path):
    # 일반적인 한국어 인코딩 시도
    encodings = ['utf-8', 'cp949', 'euc-kr']

    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as f:
                text = f.read().strip()
            return text
        except UnicodeDecodeError:
            continue

    return "텍스트 로드 실패"


def get_file_paths_npy(base_directory):
    """
    디렉토리에서 NP 파일(combined_features.npy)의 경로를 가져옵니다.
    """
    npy_files = []

    # 디렉토리 구조에 맞게 폴더 목록 생성
    base_folders = []
    for i in range(1, 19):
        if i > 9:
            base_folders.append(f"KsponSpeech_00{i}")
        else:
            base_folders.append(f"KsponSpeech_000{i}")

    # 모든 폴더 순회
    for folder in base_folders:
        folder_path = os.path.join(base_directory, folder)
        
        # 폴더가 존재하는지 확인
        if os.path.isdir(folder_path):
            print(f"폴더 검색 중: {folder_path}")
            
            # 폴더 내의 파일 검색
            for root, _, files in os.walk(folder_path):
                for file in files:
                    if file.endswith('_combined_features.npy'):
                        npy_path = os.path.join(root, file)
                        npy_files.append(npy_path)
                        if len(npy_files) % 100 == 0:  # 100개마다 로그 출력
                            print(f"찾은 파일 수: {len(npy_files)}")
        else:
            print(f"폴더가 존재하지 않음: {folder_path}")
    
    print(f"총 {len(npy_files)}개의 NP 파일을 찾았습니다.")
    return npy_files


def get_file_paths_txt(npy_files):
    """
    NP 파일 경로로부터 해당하는 텍스트 파일 경로를 생성합니다.
    """
    txt_files = []
    
    for npy_path in npy_files:
        # 파일명 추출 (경로와 확장자 제거)
        file_name = os.path.basename(npy_path).replace('_combined_features.npy', '')
        
        # 디렉토리 구조 분석
        dir_path = os.path.dirname(npy_path)
        parent_dir = os.path.basename(dir_path)
        
        # 해당 G2P 폴더 경로 생성
        g2p_dir = parent_dir + "_g2p"
        g2p_dir_path = os.path.join(os.path.dirname(dir_path), g2p_dir)
        
        # 텍스트 파일 경로 생성
        txt_path = os.path.join(g2p_dir_path, file_name + ".txt")
        
        # 파일이 존재하는지 확인 (선택적)
        if os.path.exists(txt_path):
            txt_files.append(txt_path)
        else:
            print(f"경고: 텍스트 파일이 존재하지 않습니다: {txt_path}")
    
    print(f"총 {len(txt_files)}개의 텍스트 파일을 매핑했습니다.")
    return txt_files


def create_dataset(npy_files, txt_files, max_samples=None):
    """
    NP 파일과 텍스트 파일로부터 데이터셋을 생성합니다.
    """
    data = {"audio": [], "text": []}
    
    # 최대 샘플 수 제한
    if max_samples is not None:
        npy_files = npy_files[:max_samples]
        txt_files = txt_files[:max_samples]

    for npy_file, txt_file in zip(npy_files, txt_files):
        try:
            # 파일 존재 확인
            if os.path.exists(npy_file) and os.path.exists(txt_file):
                # npy 파일 로드 테스트
                test_array = np.load(npy_file)
                if len(test_array) > 0:  # 빈 배열 체크
                    data["audio"].append(npy_file)
                    data["text"].append(load_text(txt_file))
                else:
                    print(f"빈 오디오 파일 건너뜀: {npy_file}")
            else:
                print(f"파일 없음: {npy_file} 또는 {txt_file}")
        except Exception as e:
            print(f"파일 로드 오류: {npy_file}, 오류: {e}")
            continue

    return Dataset.from_dict(data)


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: WhisperProcessor

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        batch = self.processor.feature_extractor.pad(
            {"input_features": [feature["input_features"] for feature in features]},
            return_tensors="pt"
        )

        labels_batch = self.processor.tokenizer.pad(
            {"input_ids": [feature["labels"] for feature in features]},
            return_tensors="pt"
        )

        labels = labels_batch["input_ids"].masked_fill(
            labels_batch["input_ids"] == self.processor.tokenizer.pad_token_id,
            -100
        )

        batch["labels"] = labels

        return batch


def train_whisper_model():
    # 경로 수정
    base_directory = "PreprocessData/KsponSpeech_01"
    output_dir = "whisper_finetuned"
    
    # 파일 경로 가져오기
    npy_files = get_file_paths_npy(base_directory)
    
    if len(npy_files) == 0:
        print("파일을 찾을 수 없습니다. 경로와 파일 패턴을 확인하세요.")
        return None, None
    
    txt_files = get_file_paths_txt(npy_files)
    print(f"총 {len(npy_files)}개의 오디오 파일과 {len(txt_files)}개의 텍스트 파일이 로드되었습니다.")
    
    # 파일 수가 일치하지 않으면 경고
    if len(npy_files) != len(txt_files):
        print(f"경고: 오디오 파일 수({len(npy_files)})와 텍스트 파일 수({len(txt_files)})가 일치하지 않습니다.")
        # 일치하는 파일만 사용
        if len(npy_files) > len(txt_files):
            npy_files = npy_files[:len(txt_files)]
        else:
            txt_files = txt_files[:len(npy_files)]
    
    dataset = create_dataset(npy_files, txt_files)

    def map_to_array(batch):
        arrays = []
        rates = []

        for audio_path in batch["audio"]:
            # npy 파일 로드
            audio_array = np.load(audio_path)
            # float32로 변환하고 정규화 (필요한 경우)
            if audio_array.dtype in [np.int16, np.int8]:
                max_value = float(2 ** (15 if audio_array.dtype == np.int16 else 7))
                audio_array = audio_array.astype(np.float32) / max_value
            elif audio_array.dtype != np.float32:
                audio_array = audio_array.astype(np.float32)
            
            # 샘플링 레이트는 고정 (KsponSpeech는 16kHz)
            sampling_rate = 16000
            
            arrays.append(audio_array)
            rates.append(sampling_rate)

        batch["audio"] = [{"array": arr, "sampling_rate": sr} for arr, sr in zip(arrays, rates)]
        return batch

    # 데이터셋에 Audio 형식 적용 (배치 처리)
    dataset = dataset.map(map_to_array, batched=True, batch_size=8)

    # 훈련/검증/테스트 세트 분할
    train_test_valid = dataset.train_test_split(test_size=0.2)
    test_valid = train_test_valid["test"].train_test_split(test_size=0.5)

    datasets = DatasetDict({
        "train": train_test_valid["train"],
        "test": test_valid["test"],
        "validation": test_valid["train"]
    })

    # Whisper 모델 및 프로세서 로드
    model = WhisperForConditionalGeneration.from_pretrained(
        "openai/whisper-small", 
        use_cache=False,
        low_cpu_mem_usage=True
        # torch_dtype는 자동으로 결정되도록 함
    )
    
    processor = WhisperProcessor.from_pretrained("openai/whisper-small")

    def prepare_dataset(batch):
        # 오디오 처리
        audio = batch["audio"]
    
        # 특성 추출
        input_features = processor.feature_extractor(
            audio["array"],
            sampling_rate=audio["sampling_rate"]
        ).input_features[0]
    
        # NumPy 배열을 PyTorch 텐서로 변환
        if isinstance(input_features, np.ndarray):
            input_features = torch.from_numpy(input_features)
    
        # 타입을 모델과 일치시킴 (fp16)
        input_features = input_features.to(model.dtype)
    
        # 텍스트 처리
        labels = processor.tokenizer(batch["text"]).input_ids
    
        return {
            "input_features": input_features,
            "labels": labels
        }

    # 데이터셋 전처리 적용
    processed_datasets = DatasetDict({
        split: dataset.map(
            prepare_dataset,
            remove_columns=dataset.column_names
        )
        for split, dataset in datasets.items()
    })

    # 데이터 콜레이터 설정
    data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

    def compute_metrics(pred):
        pred_ids = pred.predictions
        label_ids = pred.label_ids

        # pred_ids가 튜플인 경우 (일반적으로 첫 번째 요소가 예측값)
        if isinstance(pred_ids, tuple):
            pred_ids = pred_ids[0]

        # 이제 pred_ids가 텐서/배열인지 확인하고 3차원인 경우 처리
        if hasattr(pred_ids, 'shape') and len(pred_ids.shape) > 2:
            pred_ids = np.argmax(pred_ids, axis=-1)

        # 패딩 토큰 무시
        label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

        # 예측과 레이블을 텍스트로 디코딩
        pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
        label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

        # WER 계산
        try:
            import jiwer
            wer = jiwer.wer(label_str, pred_str)
        except ImportError:
            print("jiwer 라이브러리가 설치되지 않았습니다. WER 계산을 건너뜁니다.")
            wer = 0.0

        return {"wer": wer}

    # GPU 능력 확인하여 최적 설정 선택
    device_capability = torch.cuda.get_device_capability() if torch.cuda.is_available() else (0, 0)
    supports_bf16 = device_capability >= (8, 0)  # Ampere 이상
    
    print(f"GPU 능력: {device_capability}, BF16 지원: {supports_bf16}")
    
    # 훈련 인자 설정 (GPU에 따라 자동 선택)
    if supports_bf16:
        # BF16 사용 (Ampere GPU)
        training_args = Seq2SeqTrainingArguments(
            output_dir=output_dir,
            per_device_train_batch_size=8,
            per_device_eval_batch_size=1,
            gradient_accumulation_steps=2,
            learning_rate=1e-5,
            warmup_steps=500,
            max_steps=4000,
            gradient_checkpointing=True,
            bf16=True,
            fp16=False,
            dataloader_pin_memory=False,
            evaluation_strategy="no",
            save_steps=500,
            logging_steps=25,
            report_to=["tensorboard"],
            push_to_hub=False,
        )
    else:
        # Float32 사용 (안정적이지만 느림)
        training_args = Seq2SeqTrainingArguments(
            output_dir=output_dir,
            per_device_train_batch_size=4,  # 배치 크기 줄임
            per_device_eval_batch_size=1,
            gradient_accumulation_steps=4,  # 그래디언트 누적 늘림
            learning_rate=1e-5,
            warmup_steps=500,
            max_steps=4000,
            gradient_checkpointing=True,
            bf16=False,
            fp16=False,
            dataloader_pin_memory=False,
            evaluation_strategy="no",
            save_steps=500,
            logging_steps=25,
            report_to=["tensorboard"],
            push_to_hub=False,
        )

    # 훈련기 설정
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=processed_datasets["train"],
        data_collator=data_collator,
        tokenizer=processor.tokenizer,
    )

    # 모델 훈련
    print("모델 훈련을 시작합니다...")
    trainer.train()

    # 모델 저장
    print("모델을 저장합니다...")
    trainer.save_model(output_dir)
    processor.save_pretrained(output_dir)

    # 테스트 평가
    print("테스트 평가를 수행합니다...")
    try:
        test_results = trainer.evaluate(processed_datasets["test"])
        print(f"테스트 결과: {test_results}")
    except Exception as e:
        print(f"테스트 평가 중 오류 발생: {e}")

    return model, processor


def transcribe_audio(model, processor, audio_file):
    """
    훈련된 모델을 사용하여 오디오 파일을 텍스트로 변환합니다.
    """
    try:
        # NP 파일 로드
        audio_array = np.load(audio_file)
        
        # float32로 변환하고 정규화 (필요한 경우)
        if audio_array.dtype in [np.int16, np.int8]:
            max_value = float(2 ** (15 if audio_array.dtype == np.int16 else 7))
            audio_array = audio_array.astype(np.float32) / max_value
        elif audio_array.dtype != np.float32:
            audio_array = audio_array.astype(np.float32)
        
        # 샘플링 레이트는 고정 (KsponSpeech는 16kHz)
        sr = 16000

        # 특성 추출
        input_features = processor.feature_extractor(
            audio_array,
            sampling_rate=sr,
            return_tensors="pt"
        ).input_features
        
        # 타입을 모델과 일치시킴
        input_features = input_features.to(model.dtype)
        
        # GPU 사용하는 경우
        if torch.cuda.is_available():
            input_features = input_features.cuda()
            model = model.cuda()

        # 모델을 통한 예측
        with torch.no_grad():
            predicted_ids = model.generate(
                input_features,
                language="korean",  # 한국어 명시
                task="transcribe"
            )

        # 예측된 토큰을 텍스트로 디코딩
        transcription = processor.tokenizer.batch_decode(
            predicted_ids,
            skip_special_tokens=True
        )[0]

        return transcription
        
    except Exception as e:
        print(f"음성 인식 오류: {e}")
        return "음성 인식 실패"


if __name__ == "__main__":
    print("Whisper 한국어 파인튜닝을 시작합니다...")
    
    # 모델 훈련
    model, processor = train_whisper_model()
    
    if model is None or processor is None:
        print("모델 훈련에 실패했습니다.")
        exit()
    
    print("모델 훈련이 완료되었습니다!")
    
    # 테스트할 디렉터리 설정
    test_directory = "PreprocessData/KsponSpeech_01"
    
    # 테스트 파일 가져오기
    npy_files = get_file_paths_npy(test_directory)
    txt_files = get_file_paths_txt(npy_files)
    
    print(f"테스트할 파일: {len(npy_files)}개")
    
    # 테스트할 파일 수 제한 (선택적)
    max_test_files = 10
    test_npy_files = npy_files[:max_test_files]
    test_txt_files = txt_files[:max_test_files]
    
    print("테스트를 시작합니다...")
    
    # 모든 파일 테스트
    for i in range(len(test_npy_files)):
        test_audio = test_npy_files[i]
        reference_text = load_text(test_txt_files[i])
        
        transcription = transcribe_audio(model, processor, test_audio)
        
        print(f"파일: {os.path.basename(test_audio)}")
        print(f"원본 텍스트: {reference_text}")
        print(f"변환 결과: {transcription}")
        print("-" * 50)
    
    print("모든 작업이 완료되었습니다!")

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-26 15:06:10.895768: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-26 15:06:10.943293: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-26 15:06:11.655394: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Whisper 한국어 파인튜닝을 시작합니다...
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0001
찾은 파일 수: 100
찾은 파일 수: 200
찾은 파일 수: 300
찾은 파일 수: 400
찾은 파일 수: 500
찾은 파일 수: 600
찾은 파일 수: 700
찾은 파일 수: 800
찾은 파일 수: 900
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0002
찾은 파일 수: 1000
찾은 파일 수: 1100
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0003
찾은 파일 수: 1200
찾은 파일 수: 1300
찾은 파일 수: 1400
찾은 파일 수: 1500
찾은 파일 수: 1600
찾은 파일 수: 1700
찾은 파일 수: 1800
찾은 파일 수: 1900
찾은 파일 수: 2000
찾은 파일 수: 2100
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0004
찾은 파일 수: 2200
찾은 파일 수: 2300
찾은 파일 수: 2400
찾은 파일 수: 2500
찾은 파일 수: 2600
찾은 파일 수: 2700
찾은 파일 수: 2800
찾은 파일 수: 2900
찾은 파일 수: 3000
찾은 파일 수: 3100
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0005
찾은 파일 수: 3200
찾은 파일 수: 3300
찾은 파일 수: 3400
찾은 파일 수: 3500
찾은 파일 수: 3600
찾은 파일 수: 3700
찾은 파일 수: 3800
찾은 파일 수: 3900
찾은 파일 수: 4000
찾은 파일 수: 4100
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0006
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0007
찾은 파일 수: 4200
찾은

Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1271/1271 [04:27<00:00,  4.76 examples/s]
/usr/local/lib/python3.8/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


GPU 능력: (8, 6), BF16 지원: True


/tmp/ipykernel_24804/282252810.py:339: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
max_steps is given, it will override any value given in num_train_epochs


...

/usr/local/lib/python3.8/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is

모델을 저장합니다...
테스트 평가를 수행합니다...


테스트 결과: {'eval_loss': 3.03531551361084, 'eval_runtime': 163.0837, 'eval_samples_per_second': 7.8, 'eval_steps_per_second': 7.8, 'epoch': 6.289308176100629}
모델 훈련이 완료되었습니다!
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0001
찾은 파일 수: 100
찾은 파일 수: 200
찾은 파일 수: 300
찾은 파일 수: 400
찾은 파일 수: 500
찾은 파일 수: 600
찾은 파일 수: 700
찾은 파일 수: 800
찾은 파일 수: 900
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0002
찾은 파일 수: 1000
찾은 파일 수: 1100
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0003
찾은 파일 수: 1200
찾은 파일 수: 1300
찾은 파일 수: 1400
찾은 파일 수: 1500
찾은 파일 수: 1600
찾은 파일 수: 1700
찾은 파일 수: 1800
찾은 파일 수: 1900
찾은 파일 수: 2000
찾은 파일 수: 2100
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0004
찾은 파일 수: 2200
찾은 파일 수: 2300
찾은 파일 수: 2400
찾은 파일 수: 2500
찾은 파일 수: 2600
찾은 파일 수: 2700
찾은 파일 수: 2800
찾은 파일 수: 2900
찾은 파일 수: 3000
찾은 파일 수: 3100
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0005
찾은 파일 수: 3200
찾은 파일 수: 3300
찾은 파일 수: 3400
찾은 파일 수: 3500
찾은 파일 수: 3600
찾은 파일 수: 3700
찾은 파일 수: 3800
찾은 파일 수: 3900
찾은 파일 수: 40

You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


파일: KsponSpeech_000455_combined_features.npy
원본 텍스트: 오/ 근데 마 눤짜리 끼면 누 나파가지고. 비/
변환 결과: 오늘 러무한 사람한테는 비스태스 러무한 사람드리 비스태스 러무한 사람드리 비스태스 러무한 사람드리 비스태스 러무한 사람드리 비스태스 러무한 사람드리 비스태스 러무한 사람드리 비스태스 러무한 사람드리 비스태스 러무한 사람드리 비스태스 러무한 사람드리 비스태스 러무한 사람드리 비스태스 러무한 사람드리 비스태스 러무한 사람드리 비스태스 러무한 사람드리 비스태스 러무한 사람드리 비스태스 러무한 사람드리 비스태스 러무한 사람드리 비스태스 러무한 사람드리 비스태스 러무한 사람드리 비스태스 러무한 사람드리 비스태스 러무한 사람드리 비스탐. 비스태스 러무한 사람드리에 대해서는 비스태을 러무한 사람드리 비스태�를 러무한 사람드리에 대해서는 비스태�를 러무한 사람드리에 대해서는 비스태�를 러무한 사람드리에 대해서는 비스태��만 러무한 사람드리에 대해서는 비스태��만 러무한 사람드리에 대해서는 비스태마니르 러무한지 러무한지 러무한지 러무한지 러무한지 러무한지 러마나 러맄마나 러마나 러마나 러마나 마나 마나 마나 마나 마나 마나 마나 마나
--------------------------------------------------
파일: KsponSpeech_000336_combined_features.npy
원본 텍스트: 오/ 하, 근데, 그러케 하면 또, 보고시픈 공여니나 영화이쓰면 몯 뽀고 이러니까,
변환 결과: 오늘 래가 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 뭐지? 

In [5]:
import torch
import numpy as np

def transcribe_audio(model, processor, audio_file):
    """
    훈련된 모델을 사용하여 오디오 파일을 텍스트로 변환합니다.
    반복 생성 문제 해결 버전
    """
    try:
        # NP 파일 로드
        audio_array = np.load(audio_file)
        
        # 데이터 차원 확인 및 처리
        print(f"원본 오디오 shape: {audio_array.shape}")
        
        # 2D 배열인 경우 1D로 변환 (시간축만 남기기)
        if len(audio_array.shape) == 2:
            # 보통 (time, features) 또는 (features, time) 형태
            if audio_array.shape[0] < audio_array.shape[1]:
                audio_array = audio_array.flatten()  # 또는 적절한 축 선택
            else:
                audio_array = audio_array.flatten()  # 또는 적절한 축 선택
        
        print(f"처리된 오디오 shape: {audio_array.shape}")
        
        # float32로 변환하고 정규화
        if audio_array.dtype in [np.int16, np.int8]:
            max_value = float(2 ** (15 if audio_array.dtype == np.int16 else 7))
            audio_array = audio_array.astype(np.float32) / max_value
        elif audio_array.dtype != np.float32:
            audio_array = audio_array.astype(np.float32)
        
        # 샘플링 레이트는 고정 (KsponSpeech는 16kHz)
        sr = 16000

        # 특성 추출
        input_features = processor.feature_extractor(
            audio_array,
            sampling_rate=sr,
            return_tensors="pt"
        ).input_features
        
        # 타입을 모델과 일치시킴
        input_features = input_features.to(model.dtype)
        
        # GPU 사용하는 경우
        if torch.cuda.is_available():
            input_features = input_features.cuda()
            model = model.cuda()

        # 디버깅 정보 출력
        print(f"최종 입력 features shape: {input_features.shape}")

        # 모델을 통한 예측 (경고 해결)
        with torch.no_grad():
            predicted_ids = model.generate(
                input_features,
                max_length=448,
                num_beams=1,              # beam search 끄기
                do_sample=False,          # 샘플링 끄기  
                early_stopping=False,     # num_beams=1일 때 False로 설정
                repetition_penalty=2.0,   # 반복 패널티 강화
                no_repeat_ngram_size=3,   # 3-gram 반복 방지
                pad_token_id=processor.tokenizer.pad_token_id,
                eos_token_id=processor.tokenizer.eos_token_id,
                language="ko",            # 한국어 명시 (언어 감지 경고 해결)
                task="transcribe",        # 명시적으로 전사 작업 지정
            )

        # 디버깅 정보 출력
        print(f"생성된 토큰 수: {len(predicted_ids[0])}")
        print(f"Raw tokens (처음 20개): {predicted_ids[0][:20].tolist()}")

        # 예측된 토큰을 텍스트로 디코딩
        transcription = processor.tokenizer.batch_decode(
            predicted_ids,
            skip_special_tokens=True
        )[0]

        return transcription
        
    except Exception as e:
        print(f"음성 인식 오류: {e}")
        return "음성 인식 실패"


def transcribe_audio_simple(model, processor, audio_file):
    """
    가장 간단한 버전 - 기본 설정만 사용
    """
    try:
        # NP 파일 로드
        audio_array = np.load(audio_file)
        
        # float32로 변환하고 정규화
        if audio_array.dtype in [np.int16, np.int8]:
            max_value = float(2 ** (15 if audio_array.dtype == np.int16 else 7))
            audio_array = audio_array.astype(np.float32) / max_value
        elif audio_array.dtype != np.float32:
            audio_array = audio_array.astype(np.float32)
        
        # 특성 추출
        input_features = processor.feature_extractor(
            audio_array,
            sampling_rate=16000,
            return_tensors="pt"
        ).input_features
        
        # 타입 맞추기
        input_features = input_features.to(model.dtype)
        
        # GPU 사용
        if torch.cuda.is_available():
            input_features = input_features.cuda()
            model = model.cuda()

        # 가장 기본적인 생성
        with torch.no_grad():
            predicted_ids = model.generate(input_features)

        # 디코딩
        transcription = processor.tokenizer.batch_decode(
            predicted_ids,
            skip_special_tokens=True
        )[0]

        return transcription
        
    except Exception as e:
        print(f"음성 인식 오류: {e}")
        return "음성 인식 실패"


def transcribe_audio_advanced(model, processor, audio_file):
    """
    고급 설정 버전 - 더 세밀한 제어
    """
    try:
        # NP 파일 로드
        audio_array = np.load(audio_file)
        
        # float32로 변환하고 정규화
        if audio_array.dtype in [np.int16, np.int8]:
            max_value = float(2 ** (15 if audio_array.dtype == np.int16 else 7))
            audio_array = audio_array.astype(np.float32) / max_value
        elif audio_array.dtype != np.float32:
            audio_array = audio_array.astype(np.float32)
        
        # 특성 추출
        input_features = processor.feature_extractor(
            audio_array,
            sampling_rate=16000,
            return_tensors="pt"
        ).input_features
        
        # 타입 맞추기
        input_features = input_features.to(model.dtype)
        
        # GPU 사용
        if torch.cuda.is_available():
            input_features = input_features.cuda()
            model = model.cuda()

        # 고급 설정으로 생성
        with torch.no_grad():
            predicted_ids = model.generate(
                input_features,
                max_length=224,           # 길이 더 제한
                min_length=1,             # 최소 길이
                num_beams=2,              # beam search 활성화
                do_sample=True,           # 샘플링 활성화
                temperature=0.7,          # 온도 낮춤
                top_p=0.9,               # nucleus sampling
                repetition_penalty=1.5,   # 반복 패널티
                no_repeat_ngram_size=4,   # 4-gram 반복 방지
                early_stopping=True,
                pad_token_id=processor.tokenizer.pad_token_id,
                eos_token_id=processor.tokenizer.eos_token_id,
            )

        # 디코딩
        transcription = processor.tokenizer.batch_decode(
            predicted_ids,
            skip_special_tokens=True
        )[0]

        return transcription
        
    except Exception as e:
        print(f"음성 인식 오류: {e}")
        return "음성 인식 실패"


# 사용 예시
if __name__ == "__main__":
    # 모델 로드 (이미 훈련된 모델)
    from transformers import WhisperProcessor, WhisperForConditionalGeneration
    
    model = WhisperForConditionalGeneration.from_pretrained("whisper_finetuned")
    processor = WhisperProcessor.from_pretrained("whisper_finetuned")


        # NPZ에서 오디오 추출
    npz_data = np.load("output_npz/example_data.npz")
    audio_data = npz_data['audio_data']  # 원본 오디오
    np.save("temp_audio.npy", audio_data)
    
    # 이제 NPY 파일 사용
    test_audio = "temp_audio.npy"
    
    # 3가지 방법으로 테스트
    print("=== 기본 버전 ===")
    result1 = transcribe_audio(model, processor, test_audio)

    '''
    # 테스트 파일
    test_audio = "output_npz/example_data.wav"
    # 3가지 방법으로 테스트
    print("=== 기본 버전 ===")
    result1 = transcribe_audio(model, processor, test_audio)
    print(f"결과: {result1}")
    '''
    print("\n=== 간단 버전 ===")
    result2 = transcribe_audio_simple(model, processor, test_audio)
    print(f"결과: {result2}")
    
    print("\n=== 고급 버전 ===")
    result3 = transcribe_audio_advanced(model, processor, test_audio)
    print(f"결과: {result3}")

=== 기본 버전 ===
원본 오디오 shape: (129280,)
처리된 오디오 shape: (129280,)


You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, None], [2, 50359]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


최종 입력 features shape: torch.Size([1, 80, 3000])


Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


생성된 토큰 수: 58
Raw tokens (처음 20개): [50258, 50264, 50359, 50363, 8880, 24915, 36265, 14173, 18016, 4673, 1955, 1098, 48735, 5905, 24902, 12092, 5963, 117, 19041, 4130]

=== 간단 버전 ===
결과:  오늘 교육대토로는 특별히 청년특집으로 계획되는 거자나 청년들의 경험을 잘 듣고 여기에서 공감할 수 있는 자리가 마련되었으면 합니다.

=== 고급 버전 ===


/usr/local/lib/python3.8/dist-packages/transformers/generation/configuration_utils.py:638: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


결과:  오늘 교육대토로는 특별리 청년특집으로 계획되는 거자나요. 청년들의 경업을 잘 듣고 여기에서 공감할 수 있는 자리가 마련되 었으면 합니다


In [2]:
ls

ExampleData/            example.txt       preprocessing_delete_noise.ipynb
GPU테스트_유저03.ipynb  example_data.pcm  preprocessing_g2p.ipynb
PreprocessData/         example_data.wav  preprocessing_mfcc.ipynb
TrainData/              news_data.mp3     whisper_finetuned/
data.ipynb              news_data.pcm
dependency_library/     news_data.wav


In [ ]:
import os
import torch
import numpy as np
import librosa
import matplotlib.pyplot as plt
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import Dataset, DatasetDict, Audio
from dataclasses import dataclass
from typing import Dict, List, Union
# 5월 27일 (화) 오후 12시 시도
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

def load_text(file_path):
    # 일반적인 한국어 인코딩 시도
    encodings = ['utf-8', 'cp949', 'euc-kr']

    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as f:
                text = f.read().strip()
            return text
        except UnicodeDecodeError:
            continue

    return "텍스트 로드 실패"


def get_file_paths_npy(base_directory):
    """
    디렉토리에서 NP 파일(combined_features.npy)의 경로를 가져옵니다.
    """
    npy_files = []

    # 디렉토리 구조에 맞게 폴더 목록 생성
    base_folders = []
    for i in range(1, 19):
        if i > 9:
            base_folders.append(f"KsponSpeech_00{i}")
        else:
            base_folders.append(f"KsponSpeech_000{i}")

    # 모든 폴더 순회
    for folder in base_folders:
        folder_path = os.path.join(base_directory, folder)
        
        # 폴더가 존재하는지 확인
        if os.path.isdir(folder_path):
            print(f"폴더 검색 중: {folder_path}")
            
            # 폴더 내의 파일 검색
            for root, _, files in os.walk(folder_path):
                for file in files:
                    if file.endswith('_combined_features.npy'):
                        npy_path = os.path.join(root, file)
                        npy_files.append(npy_path)
                        if len(npy_files) % 100 == 0:  # 100개마다 로그 출력
                            print(f"찾은 파일 수: {len(npy_files)}")
        else:
            print(f"폴더가 존재하지 않음: {folder_path}")
    
    print(f"총 {len(npy_files)}개의 NP 파일을 찾았습니다.")
    return npy_files


def get_file_paths_txt(npy_files):
    """
    NP 파일 경로로부터 해당하는 텍스트 파일 경로를 생성합니다.
    """
    txt_files = []
    
    for npy_path in npy_files:
        # 파일명 추출 (경로와 확장자 제거)
        file_name = os.path.basename(npy_path).replace('_combined_features.npy', '')
        
        # 디렉토리 구조 분석
        dir_path = os.path.dirname(npy_path)
        parent_dir = os.path.basename(dir_path)
        
        # 해당 G2P 폴더 경로 생성
        g2p_dir = parent_dir + "_g2p"
        g2p_dir_path = os.path.join(os.path.dirname(dir_path), g2p_dir)
        
        # 텍스트 파일 경로 생성
        txt_path = os.path.join(g2p_dir_path, file_name + ".txt")
        
        # 파일이 존재하는지 확인 (선택적)
        if os.path.exists(txt_path):
            txt_files.append(txt_path)
        else:
            print(f"경고: 텍스트 파일이 존재하지 않습니다: {txt_path}")
    
    print(f"총 {len(txt_files)}개의 텍스트 파일을 매핑했습니다.")
    return txt_files


def create_dataset(npy_files, txt_files, max_samples=None):
    """
    NP 파일과 텍스트 파일로부터 데이터셋을 생성합니다.
    """
    data = {"audio": [], "text": []}
    
    # 최대 샘플 수 제한
    if max_samples is not None:
        npy_files = npy_files[:max_samples]
        txt_files = txt_files[:max_samples]

    for npy_file, txt_file in zip(npy_files, txt_files):
        try:
            # 파일 존재 확인
            if os.path.exists(npy_file) and os.path.exists(txt_file):
                # npy 파일 로드 테스트
                test_array = np.load(npy_file)
                if len(test_array) > 0:  # 빈 배열 체크
                    data["audio"].append(npy_file)
                    data["text"].append(load_text(txt_file))
                else:
                    print(f"빈 오디오 파일 건너뜀: {npy_file}")
            else:
                print(f"파일 없음: {npy_file} 또는 {txt_file}")
        except Exception as e:
            print(f"파일 로드 오류: {npy_file}, 오류: {e}")
            continue

    return Dataset.from_dict(data)


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: WhisperProcessor

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        batch = self.processor.feature_extractor.pad(
            {"input_features": [feature["input_features"] for feature in features]},
            return_tensors="pt"
        )

        labels_batch = self.processor.tokenizer.pad(
            {"input_ids": [feature["labels"] for feature in features]},
            return_tensors="pt"
        )

        labels = labels_batch["input_ids"].masked_fill(
            labels_batch["input_ids"] == self.processor.tokenizer.pad_token_id,
            -100
        )

        batch["labels"] = labels

        return batch


def train_whisper_model():
    # 경로 수정
    base_directory = "PreprocessData/KsponSpeech_01"
    output_dir = "whisper_finetuned"
    
    # 파일 경로 가져오기
    npy_files = get_file_paths_npy(base_directory)
    
    if len(npy_files) == 0:
        print("파일을 찾을 수 없습니다. 경로와 파일 패턴을 확인하세요.")
        return None, None
    
    txt_files = get_file_paths_txt(npy_files)
    print(f"총 {len(npy_files)}개의 오디오 파일과 {len(txt_files)}개의 텍스트 파일이 로드되었습니다.")
    
    # 파일 수가 일치하지 않으면 경고
    if len(npy_files) != len(txt_files):
        print(f"경고: 오디오 파일 수({len(npy_files)})와 텍스트 파일 수({len(txt_files)})가 일치하지 않습니다.")
        # 일치하는 파일만 사용
        if len(npy_files) > len(txt_files):
            npy_files = npy_files[:len(txt_files)]
        else:
            txt_files = txt_files[:len(npy_files)]
    
    # 데이터셋 크기 제한 (테스트용)
    max_samples = 1000  # 작은 데이터셋으로 먼저 테스트
    dataset = create_dataset(npy_files[:max_samples], txt_files[:max_samples])

    def map_to_array(batch):
        arrays = []
        rates = []

        for audio_path in batch["audio"]:
            # npy 파일 로드
            audio_array = np.load(audio_path)
            # float32로 변환하고 정규화 (필요한 경우)
            if audio_array.dtype in [np.int16, np.int8]:
                max_value = float(2 ** (15 if audio_array.dtype == np.int16 else 7))
                audio_array = audio_array.astype(np.float32) / max_value
            elif audio_array.dtype != np.float32:
                audio_array = audio_array.astype(np.float32)
            
            # 샘플링 레이트는 고정 (KsponSpeech는 16kHz)
            sampling_rate = 16000
            
            arrays.append(audio_array)
            rates.append(sampling_rate)

        batch["audio"] = [{"array": arr, "sampling_rate": sr} for arr, sr in zip(arrays, rates)]
        return batch

    # 데이터셋에 Audio 형식 적용 (배치 처리)
    dataset = dataset.map(map_to_array, batched=True, batch_size=8)

    # 훈련/검증/테스트 세트 분할
    train_test_valid = dataset.train_test_split(test_size=0.2)
    test_valid = train_test_valid["test"].train_test_split(test_size=0.5)

    datasets = DatasetDict({
        "train": train_test_valid["train"],
        "test": test_valid["test"],
        "validation": test_valid["train"]
    })

    # Whisper 모델 및 프로세서 로드
    model = WhisperForConditionalGeneration.from_pretrained(
        "openai/whisper-small", 
        use_cache=False,
        low_cpu_mem_usage=True
        # torch_dtype는 자동으로 결정되도록 함
    )
    
    processor = WhisperProcessor.from_pretrained("openai/whisper-small")

    def prepare_dataset(batch):
        # 오디오 처리
        audio = batch["audio"]
    
        # 특성 추출
        input_features = processor.feature_extractor(
            audio["array"],
            sampling_rate=audio["sampling_rate"]
        ).input_features[0]
    
        # NumPy 배열을 PyTorch 텐서로 변환
        if isinstance(input_features, np.ndarray):
            input_features = torch.from_numpy(input_features)
    
        # 타입을 모델과 일치시킴 (fp16)
        input_features = input_features.to(model.dtype)
    
        # 텍스트 처리
        labels = processor.tokenizer(batch["text"]).input_ids
    
        return {
            "input_features": input_features,
            "labels": labels
        }

    # 데이터셋 전처리 적용
    processed_datasets = DatasetDict({
        split: dataset.map(
            prepare_dataset,
            remove_columns=dataset.column_names
        )
        for split, dataset in datasets.items()
    })

    # 데이터 콜레이터 설정
    data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

    def compute_metrics(pred):
        pred_ids = pred.predictions
        label_ids = pred.label_ids

        # pred_ids가 튜플인 경우 (일반적으로 첫 번째 요소가 예측값)
        if isinstance(pred_ids, tuple):
            pred_ids = pred_ids[0]

        # 이제 pred_ids가 텐서/배열인지 확인하고 3차원인 경우 처리
        if hasattr(pred_ids, 'shape') and len(pred_ids.shape) > 2:
            pred_ids = np.argmax(pred_ids, axis=-1)

        # 패딩 토큰 무시
        label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

        # 예측과 레이블을 텍스트로 디코딩
        pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
        label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)

        # WER 계산
        try:
            import jiwer
            wer = jiwer.wer(label_str, pred_str)
        except ImportError:
            print("jiwer 라이브러리가 설치되지 않았습니다. WER 계산을 건너뜁니다.")
            wer = 0.0

        return {"wer": wer}

    # GPU 능력 확인하여 최적 설정 선택
    device_capability = torch.cuda.get_device_capability() if torch.cuda.is_available() else (0, 0)
    supports_bf16 = device_capability >= (8, 0)  # Ampere 이상
    
    print(f"GPU 능력: {device_capability}, BF16 지원: {supports_bf16}")
    
    # 훈련 인자 설정 (GPU에 따라 자동 선택)
    if supports_bf16:
        # BF16 사용 (Ampere GPU)
        training_args = Seq2SeqTrainingArguments(
            output_dir=output_dir,
            per_device_train_batch_size=8,
            per_device_eval_batch_size=1,
            gradient_accumulation_steps=2,
            learning_rate=5e-6,  # 학습률을 더 낮춤
            warmup_steps=100,   # warmup 단계 줄임
            max_steps=1000,     # 총 스텝 줄임 (테스트용)
            gradient_checkpointing=True,
            bf16=True,
            fp16=False,
            dataloader_pin_memory=False,
            evaluation_strategy="no",
            save_steps=500,
            logging_steps=25,
            report_to=["tensorboard"],
            push_to_hub=False,
        )
    else:
        # Float32 사용 (안정적이지만 느림)
        training_args = Seq2SeqTrainingArguments(
            output_dir=output_dir,
            per_device_train_batch_size=4,  # 배치 크기 줄임
            per_device_eval_batch_size=1,
            gradient_accumulation_steps=4,  # 그래디언트 누적 늘림
            learning_rate=1e-5,
            warmup_steps=500,
            max_steps=4000,
            gradient_checkpointing=True,
            bf16=False,
            fp16=False,
            dataloader_pin_memory=False,
            evaluation_strategy="no",
            save_steps=500,
            logging_steps=25,
            report_to=["tensorboard"],
            push_to_hub=False,
        )

    # 훈련기 설정
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=processed_datasets["train"],
        data_collator=data_collator,
        tokenizer=processor.tokenizer,
    )

    # 모델 훈련
    print("모델 훈련을 시작합니다...")
    trainer.train()

    # 모델 저장
    print("모델을 저장합니다...")
    trainer.save_model(output_dir)
    processor.save_pretrained(output_dir)

    # 테스트 평가
    print("테스트 평가를 수행합니다...")
    try:
        test_results = trainer.evaluate(processed_datasets["test"])
        print(f"테스트 결과: {test_results}")
    except Exception as e:
        print(f"테스트 평가 중 오류 발생: {e}")

    return model, processor


def transcribe_audio(model, processor, audio_file):
    """
    훈련된 모델을 사용하여 오디오 파일을 텍스트로 변환합니다.
    """
    try:
        # NP 파일 로드
        audio_array = np.load(audio_file)
        
        # float32로 변환하고 정규화 (필요한 경우)
        if audio_array.dtype in [np.int16, np.int8]:
            max_value = float(2 ** (15 if audio_array.dtype == np.int16 else 7))
            audio_array = audio_array.astype(np.float32) / max_value
        elif audio_array.dtype != np.float32:
            audio_array = audio_array.astype(np.float32)
        
        # 샘플링 레이트는 고정 (KsponSpeech는 16kHz)
        sr = 16000

        # 특성 추출
        input_features = processor.feature_extractor(
            audio_array,
            sampling_rate=sr,
            return_tensors="pt"
        ).input_features
        
        # 타입을 모델과 일치시킴
        input_features = input_features.to(model.dtype)
        
        # GPU 사용하는 경우
        if torch.cuda.is_available():
            input_features = input_features.cuda()
            model = model.cuda()

        # 모델을 통한 예측 (반복 방지 설정 추가)
        with torch.no_grad():
            predicted_ids = model.generate(
                input_features,
                language="korean",  # 한국어 명시
                task="transcribe",
                max_length=448,  # 최대 길이 제한
                num_beams=1,  # beam search 비활성화 (빠른 생성)
                do_sample=False,  # 샘플링 비활성화
                temperature=1.0,
                repetition_penalty=1.2,  # 반복 패널티 추가
                no_repeat_ngram_size=3,  # 3-gram 반복 방지
                early_stopping=True,
                pad_token_id=processor.tokenizer.pad_token_id,
                eos_token_id=processor.tokenizer.eos_token_id,
            )

        # 예측된 토큰을 텍스트로 디코딩
        transcription = processor.tokenizer.batch_decode(
            predicted_ids,
            skip_special_tokens=True
        )[0]

        return transcription
        
    except Exception as e:
        print(f"음성 인식 오류: {e}")
        return "음성 인식 실패"


if __name__ == "__main__":
    print("Whisper 한국어 파인튜닝을 시작합니다...")
    
    # 모델 훈련
    model, processor = train_whisper_model()
    
    if model is None or processor is None:
        print("모델 훈련에 실패했습니다.")
        exit()
    
    print("모델 훈련이 완료되었습니다!")
    
    # 테스트할 디렉터리 설정
    test_directory = "PreprocessData/KsponSpeech_01"
    
    # 테스트 파일 가져오기
    npy_files = get_file_paths_npy(test_directory)
    txt_files = get_file_paths_txt(npy_files)
    
    print(f"테스트할 파일: {len(npy_files)}개")
    
    # 테스트할 파일 수 제한 (선택적)
    max_test_files = 10
    test_npy_files = npy_files[:max_test_files]
    test_txt_files = txt_files[:max_test_files]
    
    print("테스트를 시작합니다...")
    
    # 모든 파일 테스트
    for i in range(len(test_npy_files)):
        test_audio = test_npy_files[i]
        reference_text = load_text(test_txt_files[i])
        
        transcription = transcribe_audio(model, processor, test_audio)
        
        print(f"파일: {os.path.basename(test_audio)}")
        print(f"원본 텍스트: {reference_text}")
        print(f"변환 결과: {transcription}")
        print("-" * 50)
    
    print("모든 작업이 완료되었습니다!")

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-27 03:00:55.360279: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-27 03:00:55.407883: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-27 03:00:56.128477: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Whisper 한국어 파인튜닝을 시작합니다...
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0001
찾은 파일 수: 100
찾은 파일 수: 200
찾은 파일 수: 300
찾은 파일 수: 400
찾은 파일 수: 500
찾은 파일 수: 600
찾은 파일 수: 700
찾은 파일 수: 800
찾은 파일 수: 900
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0002
찾은 파일 수: 1000
찾은 파일 수: 1100
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0003
찾은 파일 수: 1200
찾은 파일 수: 1300
찾은 파일 수: 1400
찾은 파일 수: 1500
찾은 파일 수: 1600
찾은 파일 수: 1700
찾은 파일 수: 1800
찾은 파일 수: 1900
찾은 파일 수: 2000
찾은 파일 수: 2100
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0004
찾은 파일 수: 2200
찾은 파일 수: 2300
찾은 파일 수: 2400
찾은 파일 수: 2500
찾은 파일 수: 2600
찾은 파일 수: 2700
찾은 파일 수: 2800
찾은 파일 수: 2900
찾은 파일 수: 3000
찾은 파일 수: 3100
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0005
찾은 파일 수: 3200
찾은 파일 수: 3300
찾은 파일 수: 3400
찾은 파일 수: 3500
찾은 파일 수: 3600
찾은 파일 수: 3700
찾은 파일 수: 3800
찾은 파일 수: 3900
찾은 파일 수: 4000
찾은 파일 수: 4100
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0006
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0007
찾은 파일 수: 4200
찾은

Map:  58%|████████████████████████████████████████████████████████████▎                                           | 58/100 [00:12<00:08,  4.71 examples/s]

In [47]:
import os

# 현재 디렉토리 확인
current_dir = os.getcwd()
print(f"현재 디렉토리: {current_dir}")

# 상위 디렉토리 내용 확인
parent_dir = os.path.dirname(current_dir)
print(f"상위 디렉토리: {parent_dir}")
if os.path.exists(parent_dir):
    print(f"상위 디렉토리 내용: {os.listdir(parent_dir)[:10]}")  # 처음 10개만

# TrainData 디렉토리가 있는지 확인
train_data_dir = os.path.join(current_dir, "TrainData")
print(f"TrainData 경로 존재 여부: {os.path.exists(train_data_dir)}")

# 실제 존재하는 디렉토리 목록 출력
print("\n실제 존재하는 디렉토리 탐색:")
for root, dirs, files in os.walk(current_dir, topdown=True, maxdepth=3):
    for dir_name in dirs:
        if "Kspon" in dir_name or "Speech" in dir_name or "Train" in dir_name:
            full_path = os.path.join(root, dir_name)
            print(f"발견: {full_path}")

현재 디렉토리: /data/TrainData
상위 디렉토리: /data
상위 디렉토리 내용: ['.Trash-0', 'GPU테스트_유저03.ipynb', 'TrainData', 'data.ipynb', '.ipynb_checkpoints']
TrainData 경로 존재 여부: False

실제 존재하는 디렉토리 탐색:


TypeError: walk() got an unexpected keyword argument 'maxdepth'

In [56]:
!pip install 'accelerate>=0.26.0'
!pip list | grep accelerate

accelerate                   1.0.1         


In [3]:
# PyTorch로 GPU 메모리 사용량 확인
import torch

# 현재 할당된 메모리 확인
print(f"할당된 메모리: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"캐시된 메모리: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
print(f"최대 할당된 메모리: {torch.cuda.max_memory_allocated() / 1024**3:.2f} GB")

# GPU별 메모리 통계 자세히 보기 (여러 GPU가 있는 경우)
for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
    print(f"  메모리 할당: {torch.cuda.memory_allocated(i) / 1024**3:.2f} GB")
    print(f"  메모리 예약: {torch.cuda.memory_reserved(i) / 1024**3:.2f} GB")

할당된 메모리: 10.88 GB
캐시된 메모리: 16.41 GB
최대 할당된 메모리: 18.72 GB
GPU 0: NVIDIA GeForce RTX 3090
  메모리 할당: 10.88 GB
  메모리 예약: 16.41 GB


In [24]:
# 명시적으로 모든 대형 객체 제거
import gc
import torch

# 모든 변수 확인 (옵션)
for obj in gc.get_objects():
    try:
        if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):
            print(type(obj), obj.size(), obj.device)
    except:
        pass

# 각 변수를 개별적으로 삭제
try:
    del model, processor, trainer, training_args, datasets, processed_datasets
    del dataset, train_test_valid, test_valid
except:
    pass

# 강제 GC 및 캐시 정리
gc.collect()
torch.cuda.empty_cache()

# 다시 확인
print(f"정리 후 할당된 메모리: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"정리 후 캐시된 메모리: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

/usr/local/lib/python3.8/dist-packages/torch/__init__.py:836: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  return isinstance(obj, torch.Tensor)
/tmp/ipykernel_628/1513626050.py:8: FutureWarning: `torch.distributed.reduce_op` is deprecated, please use `torch.distributed.ReduceOp` instead
  if torch.is_tensor(obj) or (hasattr(obj, 'data') and torch.is_tensor(obj.data)):


<class 'torch.Tensor'> torch.Size([]) cuda:0
<class 'torch.Tensor'> torch.Size([]) cuda:0
<class 'torch.Tensor'> torch.Size([]) cuda:0
<class 'torch.Tensor'> torch.Size([4]) cuda:0
<class 'torch.Tensor'> torch.Size([4, 50]) cuda:0
<class 'torch.Tensor'> torch.Size([200, 1500, 768]) cuda:0
<class 'torch.Tensor'> torch.Size([4, 1500, 768]) cuda:0
<class 'torch.Tensor'> torch.Size([4, 50, 51865]) cuda:0
<class 'torch.Tensor'> torch.Size([200, 138, 51865]) cuda:0
<class 'torch.Tensor'> torch.Size([204]) cuda:0
<class 'torch.Tensor'> torch.Size([200, 138]) cuda:0
<class 'torch.Tensor'> torch.Size([8, 80, 3000]) cuda:0
<class 'torch.Tensor'> torch.Size([8, 55]) cuda:0
<class 'torch.Tensor'> torch.Size([4, 80, 3000]) cuda:0
<class 'torch.Tensor'> torch.Size([4, 50]) cuda:0
<class 'torch.Tensor'> torch.Size([8, 80, 3000]) cuda:0
<class 'torch.Tensor'> torch.Size([8, 124]) cuda:0
<class 'torch.Tensor'> torch.Size([8, 80, 3000]) cpu
<class 'torch.Tensor'> torch.Size([8, 72]) cpu
<class 'torch.nn

In [19]:
import os
import torch
import numpy as np
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# PCM 파일 로드 함수
def read_pcm(file_path, sr=16000, bit_depth=16):
    """
    PCM 파일을 읽어 numpy 배열로 반환합니다.
    """
    # PCM 파일 읽기 (16비트 또는 8비트 부호 있는 정수 가정)
    if bit_depth == 16:
        raw_data = np.fromfile(file_path, dtype=np.int16)
    elif bit_depth == 8:
        raw_data = np.fromfile(file_path, dtype=np.int8)
    else:
        raw_data = np.fromfile(file_path, dtype=np.int16)  # 기본값

    # [-1, 1] 범위로 정규화
    max_value = float(2 ** (bit_depth - 1))
    normalized_data = raw_data.astype(np.float32) / max_value

    return normalized_data, sr

# 모델 추론 함수
def transcribe_audio(model, processor, audio_file, device):
    """
    훈련된 모델을 사용하여 오디오 파일을 텍스트로 변환합니다.
    """
    # PCM 파일 로드
    audio_array, sr = read_pcm(audio_file)

    # 특성 추출
    input_features = processor.feature_extractor(
        audio_array,
        sampling_rate=sr,
        return_tensors="pt"
    ).input_features
    
    # 입력 특성을 모델과 동일한 장치로 이동
    input_features = input_features.to(device)

    # 모델을 통한 예측
    with torch.no_grad():
        predicted_ids = model.generate(input_features)

    # 예측된 토큰을 텍스트로 디코딩
    transcription = processor.tokenizer.batch_decode(
        predicted_ids,
        skip_special_tokens=True
    )[0]

    return transcription

def main():
    print("=== example1.pcm 파일 테스트 ===")
    
    # 테스트할 파일 경로
    test_file = "example1.pcm"  # 이미 찾은 경로 사용
    
    # 파일 존재 확인
    if not os.path.exists(test_file):
        print(f"오류: {test_file} 파일을 찾을 수 없습니다.")
        return
    
    print(f"테스트 파일: {test_file}")
    
    # 텍스트 파일 경로 (있는 경우)
    txt_file = test_file.replace('.pcm', '.txt')
    if os.path.exists(txt_file):
        with open(txt_file, 'r', encoding='utf-8') as f:
            reference_text = f.read().strip()
        print(f"참조 텍스트: {reference_text}")
    else:
        print("참조 텍스트 파일이 없습니다.")
    
    # 기기 설정 - CUDA 문제 해결을 위한 방법
    if torch.cuda.is_available():
        device = torch.device("cuda:0")
        print("CUDA 사용 가능 - GPU 사용")
    else:
        device = torch.device("cpu")
        print("CUDA 사용 불가 - CPU 사용")
    
    # 기본 모델 로드
    print("기본 Whisper 모델을 로드합니다...")
    model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
    processor = WhisperProcessor.from_pretrained("openai/whisper-small")
    
    # 모델을 장치로 명시적 이동
    model = model.to(device)
    print(f"모델을 {device}로 이동했습니다.")
    
    # 저장된 모델 파일을 찾아 로드 시도
    model_dir = "saved_whisper_model"
    
    # PT 파일 찾기
    pt_files = []
    
    # 현재 디렉토리에서 저장된 모델 파일 찾기
    if os.path.exists(model_dir):
        for root, _, files in os.walk(model_dir):
            for file in files:
                if file.endswith('.pt'):
                    pt_files.append(os.path.join(root, file))
    
    # 다른 일반적인 위치도 확인
    for location in [".", "./saved_model", "./models"]:
        if os.path.exists(location):
            for file in os.listdir(location):
                if file.endswith('.pt'):
                    pt_files.append(os.path.join(location, file))
    
    # pt 파일 찾았다면 로드 시도
    if pt_files:
        print(f"{len(pt_files)}개의 PT 파일을 찾았습니다:")
        for i, pt_file in enumerate(pt_files):
            print(f"  {i+1}. {pt_file}")
            
        try:
            # 첫 번째 파일 로드 시도
            model_file = pt_files[0]
            print(f"모델 파일 로드 시도: {model_file}")
            
            # PT 파일을 장치에 맞게 로드
            state_dict = torch.load(model_file, map_location=device)
            model.load_state_dict(state_dict)
            print(f"모델 파일 로드 성공!")
        except Exception as e:
            print(f"모델 파일 로드 실패: {str(e)}")
            print("기본 Whisper 모델을 사용합니다.")
    else:
        print("저장된 PT 파일을 찾을 수 없습니다. 기본 Whisper 모델을 사용합니다.")
    
    # 모델을 평가 모드로 설정
    model.eval()
    
    # 음성 인식 수행
    print("\n음성 인식 중...")
    try:
        # 장치 정보를 transcribe_audio 함수에 전달
        transcription = transcribe_audio(model, processor, test_file, device)
        print("\n=== 인식 결과 ===")
        print(transcription)
        
        # 참조 텍스트가 있는 경우 WER 계산
        if os.path.exists(txt_file) and 'reference_text' in locals():
            try:
                import jiwer
                wer = jiwer.wer(reference_text, transcription)
                print(f"\nWER (Word Error Rate): {wer:.4f}")
            except ImportError:
                print("\nWER 계산을 위해 jiwer 라이브러리가 필요합니다.")
                print("pip install jiwer 명령으로 설치할 수 있습니다.")
    except Exception as e:
        print(f"음성 인식 중 오류 발생: {str(e)}")
        import traceback
        traceback.print_exc()  # 더 자세한 오류 정보 출력

if __name__ == "__main__":
    main()

=== example1.pcm 파일 테스트 ===
오류: example1.pcm 파일을 찾을 수 없습니다.


In [30]:
ls

ExampleData/            TrainData/  whisper_finetuned/
GPU테스트_유저03.ipynb  data.ipynb


In [55]:
cd /data/whisper_finetuned

/data/whisper_finetuned


In [ ]:
cd /data/whisper

In [60]:
import os
import torch
import numpy as np
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# PCM 파일 로드 함수
def read_pcm(file_path, sr=16000, bit_depth=16):
    """
    PCM 파일을 읽어 numpy 배열로 반환합니다.
    """
    # PCM 파일 읽기 (16비트 또는 8비트 부호 있는 정수 가정)
    if bit_depth == 16:
        raw_data = np.fromfile(file_path, dtype=np.int16)
    elif bit_depth == 8:
        raw_data = np.fromfile(file_path, dtype=np.int8)
    else:
        raw_data = np.fromfile(file_path, dtype=np.int16)  # 기본값

    # [-1, 1] 범위로 정규화
    max_value = float(2 ** (bit_depth - 1))
    normalized_data = raw_data.astype(np.float32) / max_value

    # 짧은 음성 샘플 출력
    print(f"PCM 파일 정보:")
    print(f"- 샘플 수: {len(normalized_data)}")
    print(f"- 샘플링 레이트: {sr}Hz")
    print(f"- 오디오 길이: {len(normalized_data)/sr:.2f}초")
    
    # 비정상적으로 짧거나 긴 경우 경고
    if len(normalized_data)/sr < 0.1:
        print("경고: 오디오가 너무 짧습니다 (0.1초 미만)")
    elif len(normalized_data)/sr > 100000:
        print("경고: 오디오가 너무 깁니다 (초과)")
    
    # 오디오 신호 통계 출력
    print(f"- 최소값: {normalized_data.min():.4f}")
    print(f"- 최대값: {normalized_data.max():.4f}")
    print(f"- 평균값: {normalized_data.mean():.4f}")
    print(f"- 표준편차: {normalized_data.std():.4f}")
    
    # 모두 0이거나 일정한 값인지 확인
    if np.all(normalized_data == 0):
        print("경고: 오디오 데이터가 모두 0입니다!")
    elif np.std(normalized_data) < 0.001:
        print("경고: 오디오 데이터의 변동이 거의 없습니다!")

    return normalized_data, sr

# 모델 추론 함수
def transcribe_audio(model, processor, audio_file, device, forced_language=None):
    """
    훈련된 모델을 사용하여 오디오 파일을 텍스트로 변환합니다.
    """
    # PCM 파일 로드
    audio_array, sr = read_pcm(audio_file)

    # 특성 추출
    input_features = processor.feature_extractor(
        audio_array,
        sampling_rate=sr,
        return_tensors="pt"
    ).input_features
    
    # 입력 특성을 모델과 동일한 장치로 이동
    input_features = input_features.to(device)

    # 생성 옵션 설정 (한국어 강제 설정)
    generation_kwargs = {}
    
    if forced_language:
        # 강제 언어 설정
        forced_decoder_ids = processor.get_decoder_prompt_ids(language=forced_language, task="transcribe")
        generation_kwargs["forced_decoder_ids"] = forced_decoder_ids
        print(f"언어를 {forced_language}로 강제 설정합니다.")

    # 모델을 통한 예측
    with torch.no_grad():
        predicted_ids = model.generate(input_features, **generation_kwargs)

    # 예측된 토큰을 텍스트로 디코딩
    transcription = processor.tokenizer.batch_decode(
        predicted_ids, 
        skip_special_tokens=True
    )[0]

    return transcription

def main():
    print("=== example1.pcm 파일 테스트 (한국어 모드) ===")
    
    # 테스트할 파일 경로
    test_file = "example1.pcm"  # 이미 찾은 경로 사용
    
    # 파일 존재 확인
    if not os.path.exists(test_file):
        print(f"오류: {test_file} 파일을 찾을 수 없습니다.")
        return
    
    print(f"테스트 파일: {test_file}")
    
    # CPU 장치 강제 설정 (CUDA 문제 회피)
    device = torch.device("cpu")
    print(f"장치: {device} (문제 해결을 위해 CPU만 사용)")
    
    # 기본 Whisper 모델 로드
    print("기본 Whisper 모델을 로드합니다...")
    model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to(device)
    processor = WhisperProcessor.from_pretrained("openai/whisper-small")
    
    # 모델 추론
    print("\n----- 언어 자동 감지 모드 -----")
    print("음성 인식 중...")
    try:
        auto_transcription = transcribe_audio(model, processor, test_file, device)
        print("\n=== 자동 언어 감지 결과 ===")
        print(auto_transcription)
    except Exception as e:
        print(f"자동 언어 모드 오류: {str(e)}")
    
    print("\n----- 한국어 강제 모드 -----")
    print("음성 인식 중...")
    try:
        # 언어를 한국어(ko)로 강제 지정
        korean_transcription = transcribe_audio(model, processor, test_file, device, forced_language="ko")
        print("\n=== 한국어 강제 모드 결과 ===")
        print(korean_transcription)
    except Exception as e:
        print(f"한국어 강제 모드 오류: {str(e)}")

if __name__ == "__main__":
    main()

=== example1.pcm 파일 테스트 (한국어 모드) ===
테스트 파일: example1.pcm
장치: cpu (문제 해결을 위해 CPU만 사용)
기본 Whisper 모델을 로드합니다...

----- 언어 자동 감지 모드 -----
음성 인식 중...
PCM 파일 정보:
- 샘플 수: 4605952
- 샘플링 레이트: 16000Hz
- 오디오 길이: 287.87초
- 최소값: -0.2336
- 최대값: 0.2339
- 평균값: 0.0000
- 표준편차: 0.0556

=== 자동 언어 감지 결과 ===
 You

----- 한국어 강제 모드 -----
음성 인식 중...
PCM 파일 정보:
- 샘플 수: 4605952
- 샘플링 레이트: 16000Hz
- 오디오 길이: 287.87초
- 최소값: -0.2336
- 최대값: 0.2339
- 평균값: 0.0000
- 표준편차: 0.0556
언어를 ko로 강제 설정합니다.

=== 한국어 강제 모드 결과 ===
 이곳은 이곳에서 가장 큰 도움이 되었습니다.


In [2]:
import os
print("whisper_finetuned 내용:")
for file in os.listdir("whisper_finetuned"):
    print(f"  {file}")

whisper_finetuned 내용:
  example1.pcm
  preprocessor_config.json
  checkpoint-2000
  whisper_finetuned
  checkpoint-3500
  tokenizer_config.json
  training_args.bin
  model.safetensors
  merges.txt
  checkpoint-3000
  .ipynb_checkpoints
  checkpoint-500
  added_tokens.json
  checkpoint-1000
  special_tokens_map.json
  checkpoint-1500
  saved_whisper_model.pt
  KsponSpeech_128001.pcm
  runs
  config.json
  normalizer.json
  checkpoint-2500
  vocab.json
  generation_config.json
  checkpoint-4000


In [ ]:
import os
import torch
import numpy as np
import librosa
from transformers import WhisperProcessor, WhisperForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import Dataset, DatasetDict
from dataclasses import dataclass
from typing import Dict, List, Union
import json

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

def load_text(file_path):
    encodings = ['utf-8', 'cp949', 'euc-kr']
    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as f:
                text = f.read().strip()
            return text
        except UnicodeDecodeError:
            continue
    return "텍스트 로드 실패"

def get_file_paths_npy(base_directory):
    npy_files = []
    base_folders = []
    for i in range(1, 19):
        if i > 9:
            base_folders.append(f"KsponSpeech_00{i}")
        else:
            base_folders.append(f"KsponSpeech_000{i}")

    for folder in base_folders:
        folder_path = os.path.join(base_directory, folder)
        if os.path.isdir(folder_path):
            print(f"폴더 검색 중: {folder_path}")
            for root, _, files in os.walk(folder_path):
                for file in files:
                    if file.endswith('_combined_features.npy'):
                        npy_path = os.path.join(root, file)
                        npy_files.append(npy_path)
                        if len(npy_files) % 100 == 0:
                            print(f"찾은 파일 수: {len(npy_files)}")
        else:
            print(f"폴더가 존재하지 않음: {folder_path}")
    
    print(f"총 {len(npy_files)}개의 NP 파일을 찾았습니다.")
    return npy_files

def get_file_paths_txt(npy_files):
    txt_files = []
    for npy_path in npy_files:
        file_name = os.path.basename(npy_path).replace('_combined_features.npy', '')
        dir_path = os.path.dirname(npy_path)
        parent_dir = os.path.basename(dir_path)
        g2p_dir = parent_dir + "_g2p"
        g2p_dir_path = os.path.join(os.path.dirname(dir_path), g2p_dir)
        txt_path = os.path.join(g2p_dir_path, file_name + ".txt")
        
        if os.path.exists(txt_path):
            txt_files.append(txt_path)
        else:
            print(f"경고: 텍스트 파일이 존재하지 않습니다: {txt_path}")
    
    print(f"총 {len(txt_files)}개의 텍스트 파일을 매핑했습니다.")
    return txt_files

def create_dataset(npy_files, txt_files, max_samples=None):
    data = {"audio": [], "text": []}
    
    if max_samples is not None:
        npy_files = npy_files[:max_samples]
        txt_files = txt_files[:max_samples]

    for npy_file, txt_file in zip(npy_files, txt_files):
        try:
            if os.path.exists(npy_file) and os.path.exists(txt_file):
                test_array = np.load(npy_file)
                if len(test_array) > 0:
                    data["audio"].append(npy_file)
                    # 발음 텍스트 그대로 저장 (정규화 X)
                    original_text = load_text(txt_file)
                    data["text"].append(original_text)
                else:
                    print(f"빈 오디오 파일 건너뜀: {npy_file}")
            else:
                print(f"파일 없음: {npy_file} 또는 {txt_file}")
        except Exception as e:
            print(f"파일 로드 오류: {npy_file}, 오류: {e}")
            continue

    return Dataset.from_dict(data)

def extend_tokenizer_vocabulary(processor, texts):
    """
    발음 기준 텍스트에 맞춰 토크나이저 어휘 확장
    """
    print("토크나이저 어휘 확장 중...")
    
    # 기존 어휘 크기
    original_vocab_size = len(processor.tokenizer)
    print(f"기존 어휘 크기: {original_vocab_size}")
    
    # 발음 텍스트에서 새로운 토큰 추출
    new_tokens = set()
    for text in texts[:1000]:  # 샘플링해서 확인
        # 발음 기호들 추출
        if "/" in text:
            new_tokens.update(["/"])
        if "+" in text:
            new_tokens.update(["+"])
        # 기타 특수 기호들...
    
    # 새 토큰 추가
    new_tokens = list(new_tokens - set(processor.tokenizer.get_vocab().keys()))
    if new_tokens:
        print(f"새로 추가할 토큰: {new_tokens}")
        processor.tokenizer.add_tokens(new_tokens)
        print(f"확장된 어휘 크기: {len(processor.tokenizer)}")
    
    return processor

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: WhisperProcessor

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        batch = self.processor.feature_extractor.pad(
            {"input_features": [feature["input_features"] for feature in features]},
            return_tensors="pt"
        )

        labels_batch = self.processor.tokenizer.pad(
            {"input_ids": [feature["labels"] for feature in features]},
            return_tensors="pt"
        )

        labels = labels_batch["input_ids"].masked_fill(
            labels_batch["input_ids"] == self.processor.tokenizer.pad_token_id,
            -100
        )

        batch["labels"] = labels
        return batch

def train_whisper_pronunciation_model():
    base_directory = "PreprocessData/KsponSpeech_01"
    output_dir = "whisper_pronunciation_finetuned"
    
    # 파일 경로 가져오기
    npy_files = get_file_paths_npy(base_directory)
    
    if len(npy_files) == 0:
        print("파일을 찾을 수 없습니다.")
        return None, None
    
    txt_files = get_file_paths_txt(npy_files)
    print(f"총 {len(npy_files)}개의 오디오 파일과 {len(txt_files)}개의 텍스트 파일이 로드되었습니다.")
    
    # 파일 수 맞추기
    if len(npy_files) != len(txt_files):
        print(f"경고: 파일 수 불일치")
        min_len = min(len(npy_files), len(txt_files))
        npy_files = npy_files[:min_len]
        txt_files = txt_files[:min_len]
    
    dataset = create_dataset(npy_files, txt_files)

    def map_to_array(batch):
        arrays = []
        rates = []

        for audio_path in batch["audio"]:
            audio_array = np.load(audio_path)
            if audio_array.dtype in [np.int16, np.int8]:
                max_value = float(2 ** (15 if audio_array.dtype == np.int16 else 7))
                audio_array = audio_array.astype(np.float32) / max_value
            elif audio_array.dtype != np.float32:
                audio_array = audio_array.astype(np.float32)
            
            sampling_rate = 16000
            arrays.append(audio_array)
            rates.append(sampling_rate)

        batch["audio"] = [{"array": arr, "sampling_rate": sr} for arr, sr in zip(arrays, rates)]
        return batch

    dataset = dataset.map(map_to_array, batched=True, batch_size=8)

    # 데이터셋 분할
    train_test_valid = dataset.train_test_split(test_size=0.2)
    test_valid = train_test_valid["test"].train_test_split(test_size=0.5)

    datasets = DatasetDict({
        "train": train_test_valid["train"],
        "test": test_valid["test"],
        "validation": test_valid["train"]
    })

    # 모델 및 프로세서 로드
    model = WhisperForConditionalGeneration.from_pretrained(
        "openai/whisper-small", 
        use_cache=False,
        low_cpu_mem_usage=True
    )
    
    processor = WhisperProcessor.from_pretrained("openai/whisper-small")
    
    # 🎯 핵심: 토크나이저 어휘 확장
    all_texts = [item["text"] for item in dataset]
    processor = extend_tokenizer_vocabulary(processor, all_texts)
    
    # 모델 임베딩 크기 조정 (새 토큰 추가 시)
    if len(processor.tokenizer) > model.config.vocab_size:
        print("모델 임베딩 크기 조정 중...")
        model.resize_token_embeddings(len(processor.tokenizer))

    def prepare_dataset(batch):
        audio = batch["audio"]
        
        input_features = processor.feature_extractor(
            audio["array"],
            sampling_rate=audio["sampling_rate"]
        ).input_features[0]
        
        if isinstance(input_features, np.ndarray):
            input_features = torch.from_numpy(input_features)
        
        input_features = input_features.to(model.dtype)
        
        # 🎯 핵심: 발음 텍스트 그대로 토크나이징 (정규화 없음)
        labels = processor.tokenizer(
            batch["text"],
            truncation=True,
            max_length=448,
            padding=False
        ).input_ids
        
        return {
            "input_features": input_features,
            "labels": labels
        }

    processed_datasets = DatasetDict({
        split: dataset.map(
            prepare_dataset,
            remove_columns=dataset.column_names
        )
        for split, dataset in datasets.items()
    })

    data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

    # GPU 능력 확인
    device_capability = torch.cuda.get_device_capability() if torch.cuda.is_available() else (0, 0)
    supports_bf16 = device_capability >= (8, 0)
    
    print(f"GPU 능력: {device_capability}, BF16 지원: {supports_bf16}")
    
    # 훈련 설정
    if supports_bf16:
        training_args = Seq2SeqTrainingArguments(
            output_dir=output_dir,
            per_device_train_batch_size=8,
            per_device_eval_batch_size=1,
            gradient_accumulation_steps=2,
            learning_rate=5e-6,  # 더 낮은 학습률 (발음 학습용)
            warmup_steps=500,
            max_steps=6000,      # 더 많은 스텝
            gradient_checkpointing=True,
            bf16=True,
            fp16=False,
            dataloader_pin_memory=False,
            evaluation_strategy="no",
            save_steps=500,
            logging_steps=25,
            report_to=["tensorboard"],
            push_to_hub=False,
        )
    else:
        training_args = Seq2SeqTrainingArguments(
            output_dir=output_dir,
            per_device_train_batch_size=4,
            per_device_eval_batch_size=1,
            gradient_accumulation_steps=4,
            learning_rate=5e-6,
            warmup_steps=500,
            max_steps=6000,
            gradient_checkpointing=True,
            bf16=False,
            fp16=False,
            dataloader_pin_memory=False,
            evaluation_strategy="no",
            save_steps=500,
            logging_steps=25,
            report_to=["tensorboard"],
            push_to_hub=False,
        )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=processed_datasets["train"],
        data_collator=data_collator,
        tokenizer=processor.tokenizer,
    )

    print("발음 기준 모델 훈련을 시작합니다...")
    trainer.train()

    print("모델을 저장합니다...")
    trainer.save_model(output_dir)
    processor.save_pretrained(output_dir)

    # 토크나이저 설정도 별도 저장
    with open(os.path.join(output_dir, "pronunciation_config.json"), "w") as f:
        json.dump({
            "original_vocab_size": model.config.vocab_size,
            "extended_vocab_size": len(processor.tokenizer),
            "training_type": "pronunciation_based"
        }, f, indent=2)

    return model, processor

# 🎯 발음 기준 추론 함수
def transcribe_audio_pronunciation(model, processor, audio_file):
    try:
        audio_array = np.load(audio_file)
        
        if audio_array.dtype in [np.int16, np.int8]:
            max_value = float(2 ** (15 if audio_array.dtype == np.int16 else 7))
            audio_array = audio_array.astype(np.float32) / max_value
        elif audio_array.dtype != np.float32:
            audio_array = audio_array.astype(np.float32)
        
        sr = 16000
        input_features = processor.feature_extractor(
            audio_array,
            sampling_rate=sr,
            return_tensors="pt"
        ).input_features
        
        input_features = input_features.to(model.dtype)
        
        if torch.cuda.is_available():
            input_features = input_features.cuda()
            model = model.cuda()

        # 🎯 발음 기준 생성 (최소한의 제약)
        with torch.no_grad():
            predicted_ids = model.generate(
                input_features,
                max_length=448,
                num_beams=1,
                do_sample=False,
                # language, task 파라미터 제거 (발음 출력 유도)
            )

        transcription = processor.tokenizer.batch_decode(
            predicted_ids,
            skip_special_tokens=True
        )[0]

        return transcription
        
    except Exception as e:
        print(f"음성 인식 오류: {e}")
        return "음성 인식 실패"

if __name__ == "__main__":
    print("발음 기준 Whisper 파인튜닝을 시작합니다...")
    
    model, processor = train_whisper_pronunciation_model()
    
    if model is None or processor is None:
        print("모델 훈련에 실패했습니다.")
        exit()
    
    print("발음 기준 모델 훈련이 완료되었습니다!")
    
    # 테스트
    test_directory = "PreprocessData/KsponSpeech_01"
    npy_files = get_file_paths_npy(test_directory)
    txt_files = get_file_paths_txt(npy_files)
    
    max_test_files = 5
    test_npy_files = npy_files[:max_test_files]
    test_txt_files = txt_files[:max_test_files]
    
    print("발음 기준 테스트를 시작합니다...")
    
    for i in range(len(test_npy_files)):
        test_audio = test_npy_files[i]
        reference_text = load_text(test_txt_files[i])
        
        transcription = transcribe_audio_pronunciation(model, processor, test_audio)
        
        print(f"파일: {os.path.basename(test_audio)}")
        print(f"원본 텍스트: {reference_text}")
        print(f"발음 변환 결과: {transcription}")
        print("-" * 50)
    
    print("모든 작업이 완료되었습니다!")

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-27 15:15:46.117660: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-27 15:15:46.165587: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-27 15:15:46.959438: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


발음 기준 Whisper 파인튜닝을 시작합니다...
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0001
찾은 파일 수: 100
찾은 파일 수: 200
찾은 파일 수: 300
찾은 파일 수: 400
찾은 파일 수: 500
찾은 파일 수: 600
찾은 파일 수: 700
찾은 파일 수: 800
찾은 파일 수: 900
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0002
찾은 파일 수: 1000
찾은 파일 수: 1100
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0003
찾은 파일 수: 1200
찾은 파일 수: 1300
찾은 파일 수: 1400
찾은 파일 수: 1500
찾은 파일 수: 1600
찾은 파일 수: 1700
찾은 파일 수: 1800
찾은 파일 수: 1900
찾은 파일 수: 2000
찾은 파일 수: 2100
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0004
찾은 파일 수: 2200
찾은 파일 수: 2300
찾은 파일 수: 2400
찾은 파일 수: 2500
찾은 파일 수: 2600
찾은 파일 수: 2700
찾은 파일 수: 2800
찾은 파일 수: 2900
찾은 파일 수: 3000
찾은 파일 수: 3100
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0005
찾은 파일 수: 3200
찾은 파일 수: 3300
찾은 파일 수: 3400
찾은 파일 수: 3500
찾은 파일 수: 3600
찾은 파일 수: 3700
찾은 파일 수: 3800
찾은 파일 수: 3900
찾은 파일 수: 4000
찾은 파일 수: 4100
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0006
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0007
찾은 파일 수: 4200


Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 12713/12713 [00:05<00:00, 2169.01 examples/s]


토크나이저 어휘 확장 중...
기존 어휘 크기: 51865


Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 1271/1271 [04:27<00:00,  4.75 examples/s]
/usr/local/lib/python3.8/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


...

/usr/local/lib/python3.8/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
25,9.331400
50,8.542900
75,7.546800
100,6.848900
125,5.999500
150,5.666000
175,5.354600
200,5.164900
225,4.945200
250,4.821700


/usr/local/lib/python3.8/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 4183, 4667, 6585, 6647, 7273, 9061, 9383, 10428, 10929, 11938, 12033, 12331, 12562, 13793, 14157, 14635, 15265, 15618, 16553, 16604, 18362, 18956, 20075, 21675, 22520, 26130, 26161, 26435, 28279, 29464, 31650, 32302, 32470, 36865, 42863, 47425, 49870, 50254, 50258, 50360, 50361, 50362], 'begin_suppress_tokens': [220, 50257]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is

모델을 저장합니다...
발음 기준 모델 훈련이 완료되었습니다!
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0001
찾은 파일 수: 100
찾은 파일 수: 200
찾은 파일 수: 300
찾은 파일 수: 400
찾은 파일 수: 500
찾은 파일 수: 600
찾은 파일 수: 700
찾은 파일 수: 800
찾은 파일 수: 900
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0002
찾은 파일 수: 1000
찾은 파일 수: 1100
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0003
찾은 파일 수: 1200
찾은 파일 수: 1300
찾은 파일 수: 1400
찾은 파일 수: 1500
찾은 파일 수: 1600
찾은 파일 수: 1700
찾은 파일 수: 1800
찾은 파일 수: 1900
찾은 파일 수: 2000
찾은 파일 수: 2100
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0004
찾은 파일 수: 2200
찾은 파일 수: 2300
찾은 파일 수: 2400
찾은 파일 수: 2500
찾은 파일 수: 2600
찾은 파일 수: 2700
찾은 파일 수: 2800
찾은 파일 수: 2900
찾은 파일 수: 3000
찾은 파일 수: 3100
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0005
찾은 파일 수: 3200
찾은 파일 수: 3300
찾은 파일 수: 3400
찾은 파일 수: 3500
찾은 파일 수: 3600
찾은 파일 수: 3700
찾은 파일 수: 3800
찾은 파일 수: 3900
찾은 파일 수: 4000
찾은 파일 수: 4100
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0006
폴더 검색 중: PreprocessData/KsponSpeech_01/KsponSpeech_0007
찾은 파일 수:

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
/usr/local/lib/python3.8/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [13]:
import os
import torch
import librosa
import soundfile as sf
import numpy as np
from transformers import WhisperProcessor, WhisperForConditionalGeneration

def load_trained_model(model_path="whisper_pronunciation_finetuned"):
    """저장된 발음 모델 불러오기"""
    print(f"🔄 모델 로딩 중: {model_path}")
    
    if not os.path.exists(model_path):
        print(f"❌ 모델 폴더가 없습니다: {model_path}")
        return None, None
    
    try:
        model = WhisperForConditionalGeneration.from_pretrained(model_path)
        processor = WhisperProcessor.from_pretrained(model_path)
        
        if torch.cuda.is_available():
            model = model.cuda()
            print("🔥 GPU 사용 중")
        else:
            print("💻 CPU 사용 중")
            
        print("✅ 모델 로드 완료!")
        return model, processor
        
    except Exception as e:
        print(f"❌ 모델 로드 실패: {e}")
        return None, None

def transcribe_real_audio(model, processor, audio_file_path):
    """실제 오디오 파일(.wav, .mp3 등)을 발음으로 변환"""
    
    print(f"🎵 오디오 파일 처리 중: {os.path.basename(audio_file_path)}")
    
    try:
        # 1. 오디오 파일 로드 (Whisper가 하는 방식 그대로)
        audio, sr = librosa.load(audio_file_path, sr=16000)  # 16kHz로 리샘플링
        print(f"   📊 오디오 길이: {len(audio)/sr:.2f}초")
        print(f"   📊 샘플링 레이트: {sr}Hz")
        print(f"   📊 오디오 범위: [{audio.min():.3f}, {audio.max():.3f}]")
        
        # 2. Whisper 특징 추출 (일반 추론과 동일)
        input_features = processor(
            audio, 
            sampling_rate=sr, 
            return_tensors="pt"
        ).input_features
        
        print(f"   📊 입력 특징 크기: {input_features.shape}")
        
        # 3. GPU로 이동
        if torch.cuda.is_available():
            input_features = input_features.cuda()
        
        # 4. 발음 기준 추론
        with torch.no_grad():
            predicted_ids = model.generate(
                input_features,
                max_length=448,
                num_beams=1,
                do_sample=False,
                # language="ko"  # 필요시 추가
            )
        
        # 5. 텍스트 디코딩
        transcription = processor.tokenizer.batch_decode(
            predicted_ids, 
            skip_special_tokens=True
        )[0]
        
        print(f"   🤖 발음 변환 결과: {transcription}")
        return transcription
        
    except Exception as e:
        print(f"   ❌ 처리 실패: {e}")
        return None

def test_with_sample_audio():
    """샘플 오디오로 테스트"""
    
    print("🎯 실제 오디오 파일 -> 발음 변환 테스트")
    print("=" * 60)
    
    # 모델 로드
    model, processor = load_trained_model()
    if model is None:
        return
    
    # 테스트할 오디오 파일들 찾기
    audio_extensions = ['.wav', '.mp3', '.flac', '.m4a']
    test_files = []
    
    # 현재 디렉토리와 하위 폴더에서 오디오 파일 찾기
    for root, dirs, files in os.walk('.'):
        for file in files:
            if any(file.lower().endswith(ext) for ext in audio_extensions):
                audio_path = os.path.join(root, file)
                test_files.append(audio_path)
                if len(test_files) >= 5:  # 최대 5개
                    break
        if len(test_files) >= 5:
            break
    
    if not test_files:
        print("❌ 테스트할 오디오 파일을 찾을 수 없습니다!")
        print("다음 형식의 파일을 현재 폴더에 넣어주세요: .wav, .mp3, .flac, .m4a")
        
        # 간단한 테스트 음성 생성 (선택사항)
        create_test_audio = input("테스트용 음성 파일을 생성하시겠습니까? (y/n): ")
        if create_test_audio.lower() == 'y':
            create_sample_audio()
            return
        else:
            return
    
    print(f"📁 {len(test_files)}개의 오디오 파일 발견")
    print()
    
    # 각 파일에 대해 추론 실행
    for i, audio_file in enumerate(test_files, 1):
        print(f"🔍 테스트 {i}/{len(test_files)}")
        result = transcribe_real_audio(model, processor, audio_file)
        
        if result:
            print("   ✅ 성공!")
        else:
            print("   ❌ 실패!")
        
        print("-" * 50)

def create_sample_audio():
    """테스트용 샘플 오디오 생성"""
    try:
        import numpy as np
        
        print("🔊 테스트용 음성 파일 생성 중...")
        
        # 간단한 사인파 생성 (440Hz, 2초)
        sr = 16000
        duration = 2.0
        t = np.linspace(0, duration, int(sr * duration))
        
        # 440Hz 톤 (A4 음)
        audio = 0.3 * np.sin(2 * np.pi * 440 * t)
        
        # 파일 저장
        output_file = "test_audio.wav"
        sf.write(output_file, audio, sr)
        
        print(f"✅ 테스트 파일 생성 완료: {output_file}")
        print("이제 다시 테스트를 실행해보세요!")
        
    except ImportError:
        print("❌ soundfile 라이브러리가 필요합니다: pip install soundfile")
    except Exception as e:
        print(f"❌ 테스트 파일 생성 실패: {e}")

def test_single_file():
    """단일 파일 직접 지정해서 테스트"""
    
    print("📂 단일 파일 테스트")
    
    # 파일 경로 직접 입력
    audio_file = input("오디오 파일 경로를 입력하세요: ").strip()
    
    if not os.path.exists(audio_file):
        print(f"❌ 파일이 존재하지 않습니다: {audio_file}")
        return
    
    # 모델 로드
    model, processor = load_trained_model()
    if model is None:
        return
    
    # 추론 실행
    print()
    result = transcribe_real_audio(model, processor, audio_file)
    
    if result:
        print("\n🎉 추론 성공!")
        print(f"결과: {result}")
    else:
        print("\n❌ 추론 실패!")

def load_pcm_file(pcm_file_path, sample_rate=16000):
    """PCM 파일을 로드하여 numpy 배열로 변환 (사용자 전처리 방식 적용)"""
    try:
        # PCM 파일을 16비트 정수로 읽기
        with open(pcm_file_path, 'rb') as f:
            pcm_data = f.read()
        
        # bytes를 numpy 배열로 변환 (16비트 정수 가정)
        np_pcm = np.frombuffer(pcm_data, dtype=np.int16)
        
        print(f"   📊 원본 PCM 데이터 길이: {len(np_pcm)} 샘플")
        print(f"   📊 원본 데이터 범위: [{np_pcm.min()}, {np_pcm.max()}]")
        
        # 🎯 사용자 전처리 방식 적용
        normalized = np_pcm / np.max(np.abs(np_pcm))
        
        print(f"   📊 정규화 후 범위: [{normalized.min():.3f}, {normalized.max():.3f}]")
        print(f"   📊 오디오 길이: {len(normalized)/sample_rate:.2f}초")
        
        return normalized.astype(np.float32)
        
    except Exception as e:
        print(f"   ❌ PCM 파일 로드 실패: {e}")
        return None

def transcribe_pcm_file(model, processor, pcm_file_path):
    """PCM 파일을 발음으로 변환"""
    
    print(f"🎵 PCM 파일 처리 중: {os.path.basename(pcm_file_path)}")
    
    try:
        # 1. PCM 파일 로드
        audio = load_pcm_file(pcm_file_path)
        if audio is None:
            return None
        
        # 2. Whisper 특징 추출
        input_features = processor(
            audio, 
            sampling_rate=16000, 
            return_tensors="pt"
        ).input_features
        
        print(f"   📊 입력 특징 크기: {input_features.shape}")
        
        # 3. GPU로 이동
        if torch.cuda.is_available():
            input_features = input_features.cuda()
        
        # 4. 발음 기준 추론
        with torch.no_grad():
            predicted_ids = model.generate(
                input_features,
                max_length=448,
                num_beams=1,
                do_sample=False,
            )
        
        # 5. 텍스트 디코딩
        transcription = processor.tokenizer.batch_decode(
            predicted_ids, 
            skip_special_tokens=True
        )[0]
        
        print(f"   🤖 발음 변환 결과: {transcription}")
        return transcription
        
    except Exception as e:
        print(f"   ❌ 처리 실패: {e}")
        return None

def main():
    """메인 함수 - example_data.pcm 파일 처리"""
    
    print("🎤 PCM 파일 발음 변환 테스트")
    print("=" * 60)
    
    # PCM 파일 경로
    pcm_file = "000025.wav"
    
    # 파일 존재 확인
    if not os.path.exists(pcm_file):
        print(f"❌ PCM 파일이 없습니다: {pcm_file}")
        print("example_data.pcm 파일을 현재 폴더에 넣어주세요!")
        return
    
    # 모델 로드
    model, processor = load_trained_model()
    if model is None:
        print("❌ 모델 로드 실패!")
        return
    
    print(f"📁 처리할 파일: {pcm_file}")
    print()
    
    # PCM 파일 추론 실행
    result = transcribe_pcm_file(model, processor, pcm_file)
    
    print()
    print("=" * 60)
    if result:
        print("🎉 PCM -> 발음 변환 성공!")
        print(f"📝 최종 결과: {result}")
    else:
        print("❌ PCM -> 발음 변환 실패!")
    print("=" * 60)

if __name__ == "__main__":
    main()

🎤 PCM 파일 발음 변환 테스트
🔄 모델 로딩 중: whisper_pronunciation_finetuned
🔥 GPU 사용 중
✅ 모델 로드 완료!
📁 처리할 파일: 000025.wav

🎵 PCM 파일 처리 중: 000025.wav
   📊 원본 PCM 데이터 길이: 193515 샘플
   📊 원본 데이터 범위: [-7886, 32000]
   📊 정규화 후 범위: [-0.246, 1.000]
   📊 오디오 길이: 12.09초
   📊 입력 특징 크기: torch.Size([1, 80, 3000])
   🤖 발음 변환 결과:  거기서 아무런 그런 개입이 업써 관찰만 수천 명의 아이드를 쭉 크는 걸 보니까 부모한테 맨처 맨날 애착을 하죠. 그래서 부모한테 맨날 애착을 하죠. 그래서 부모한테 맨날 애착을 하죠. 그래서 부모한테 맨날 애착을 하죠. 그래서 부모한테 맨날 애착을 하죠. 그래서 부모한테 맨날 애착을 하죠. 그래서 부모한테 맨날 애착을 하죠. 그래서 부모한테 맨날 애착을 하죠. 그래서 부모한테 맨날 애착을 하죠. 그래서 부모한테 맨날 애착을 하죠. 그래서 부모한테 맨날 애착을 하죠. 그래서 부모한테 맨날 애착을 하죠. 그래서 부모한테 맨날 애착을 하죠. 그래서 부모한테 맨날 애착을 하죠. 그래서 부모한테 맨날 애착을 하죠. 그래서 부모한테 맨날 애착을 하죠. 그래서 부모한테 맨날 애착을 하죠. 그래서 부모한테 맨날 애착을 하죠. 그래서 부모한테 맨날 애착을 하죠. 그래서 부모한테 맨날 애착을 하죠. 그래서 부모한테 맨날 애착을 하죠. 그래서 부모한테 맨날 애착을 하죠. 그래서 부모한테 맨날 애착을 하죠. 그래서 부모한테 맨날 애착을 하죠. 그래서 부모한테 맨들 쑤맨들 하죠. 그래서 부모한테 맨들 쑤맨들 하죠. 그래서 부모한테 맨들 쑨들 하죠. 부모니깨�라고. 부모니깨�라고. 부모니깨�맨�라고. 부모니깨�맨�맨�맨�맨�맨�맨�맨�맨��

🎉 PCM -> 발음 변환 성공!
📝 최종 결과:  거기서 아무런 그런 개입이 업써 관찰만 수천 명의 아이드를 쭉

In [14]:
import os
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor

def convert_transformers_to_pt():
    """Transformers 모델을 .pt 파일로 변환"""
    
    print("🔄 Transformers 모델을 .pt 파일로 변환")
    print("=" * 60)
    
    # 입력 폴더 (Transformers 모델이 저장된 곳)
    input_model_path = "whisper_pronunciation_finetuned"
    
    # 출력 파일명
    output_pt_file = "whisper_pronunciation_model.pt"
    
    # 모델 폴더 확인
    if not os.path.exists(input_model_path):
        print(f"❌ 모델 폴더가 없습니다: {input_model_path}")
        return
    
    # 필요한 파일들 확인
    required_files = ["config.json"]
    model_file = None
    
    # 모델 파일 찾기 (safetensors 또는 bin)
    if os.path.exists(os.path.join(input_model_path, "model.safetensors")):
        model_file = "model.safetensors"
    elif os.path.exists(os.path.join(input_model_path, "pytorch_model.bin")):
        model_file = "pytorch_model.bin"
    
    if model_file:
        required_files.append(model_file)
    
    print(f"📋 필요한 파일들 확인:")
    for file in required_files:
        file_path = os.path.join(input_model_path, file)
        if os.path.exists(file_path):
            print(f"   ✅ {file}")
        else:
            print(f"   ❌ {file} - 없음")
            return
    
    try:
        # 1. 모델과 프로세서 로드
        print("\n🔄 모델 로딩 중...")
        model = WhisperForConditionalGeneration.from_pretrained(input_model_path)
        processor = WhisperProcessor.from_pretrained(input_model_path)
        
        print("✅ 모델 로드 완료!")
        
        # 2. 모델 정보 출력
        print(f"📊 모델 정보:")
        print(f"   - 모델 타입: {type(model).__name__}")
        print(f"   - 어휘 크기: {len(processor.tokenizer)}")
        print(f"   - 모델 크기: {sum(p.numel() for p in model.parameters()):,} parameters")
        
        # 3. .pt 파일로 저장할 데이터 준비
        save_data = {
            'model_state_dict': model.state_dict(),
            'model_config': model.config.to_dict(),
            'tokenizer': processor.tokenizer,
            'feature_extractor': processor.feature_extractor,
            'processor_config': {
                'tokenizer_class': processor.tokenizer.__class__.__name__,
                'feature_extractor_class': processor.feature_extractor.__class__.__name__,
            },
            'training_info': {
                'model_type': 'whisper_pronunciation_finetuned',
                'base_model': 'openai/whisper-small',
                'task': 'pronunciation_transcription'
            }
        }
        
        # 4. .pt 파일로 저장
        print(f"\n💾 .pt 파일로 저장 중: {output_pt_file}")
        torch.save(save_data, output_pt_file)
        
        # 5. 저장된 파일 크기 확인
        file_size = os.path.getsize(output_pt_file) / (1024 * 1024)  # MB
        print(f"✅ 저장 완료!")
        print(f"📊 파일 크기: {file_size:.1f} MB")
        
        # 6. 검증을 위한 로드 테스트
        print(f"\n🔍 저장된 파일 검증 중...")
        loaded_data = torch.load(output_pt_file, map_location='cpu')
        
        print("✅ 검증 완료! 포함된 데이터:")
        for key in loaded_data.keys():
            print(f"   - {key}")
        
        return output_pt_file
        
    except Exception as e:
        print(f"❌ 변환 실패: {e}")
        return None

def load_pt_model(pt_file_path):
    """저장된 .pt 파일에서 모델 로드"""
    
    print(f"🔄 .pt 파일에서 모델 로딩: {pt_file_path}")
    
    try:
        # .pt 파일 로드
        checkpoint = torch.load(pt_file_path, map_location='cpu')
        
        # 모델 생성
        from transformers import WhisperConfig
        config = WhisperConfig.from_dict(checkpoint['model_config'])
        model = WhisperForConditionalGeneration(config)
        
        # 모델 가중치 로드
        model.load_state_dict(checkpoint['model_state_dict'])
        
        # 프로세서 복원
        tokenizer = checkpoint['tokenizer']
        feature_extractor = checkpoint['feature_extractor']
        processor = WhisperProcessor(feature_extractor=feature_extractor, tokenizer=tokenizer)
        
        print("✅ .pt 파일에서 모델 로드 완료!")
        
        return model, processor
        
    except Exception as e:
        print(f"❌ .pt 파일 로드 실패: {e}")
        return None, None

def test_pt_model():
    """변환된 .pt 모델 테스트"""
    
    print("🧪 변환된 .pt 모델 테스트")
    print("=" * 60)
    
    pt_file = "whisper_pronunciation_model.pt"
    
    if not os.path.exists(pt_file):
        print(f"❌ .pt 파일이 없습니다: {pt_file}")
        return
    
    # .pt 파일에서 모델 로드
    model, processor = load_pt_model(pt_file)
    
    if model is None:
        return
    
    # GPU 사용 가능하면 이동
    if torch.cuda.is_available():
        model = model.cuda()
        print("🔥 GPU로 모델 이동 완료")
    
    # 간단한 추론 테스트 (더미 데이터)
    print("🔍 더미 데이터로 추론 테스트...")
    
    try:
        # 더미 오디오 입력 생성
        import numpy as np
        dummy_audio = np.random.randn(16000).astype(np.float32)  # 1초 더미 오디오
        
        # 특징 추출
        input_features = processor(
            dummy_audio,
            sampling_rate=16000,
            return_tensors="pt"
        ).input_features
        
        if torch.cuda.is_available():
            input_features = input_features.cuda()
        
        # 추론
        with torch.no_grad():
            outputs = model.generate(
                input_features,
                max_length=50,
                num_beams=1,
                do_sample=False
            )
        
        result = processor.tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        print(f"✅ 추론 테스트 성공!")
        print(f"📝 더미 결과: {result}")
        
    except Exception as e:
        print(f"❌ 추론 테스트 실패: {e}")

def main():
    """메인 함수"""
    
    print("🔧 Whisper 모델 .pt 변환 도구")
    print("=" * 80)
    
    # 1. Transformers 모델을 .pt로 변환
    pt_file = convert_transformers_to_pt()
    
    if pt_file:
        print("\n" + "=" * 80)
        
        # 2. 변환된 모델 테스트
        test_pt_model()
        
        print("\n" + "=" * 80)
        print("🎉 변환 완료!")
        print(f"📁 사용할 파일: {pt_file}")
        print("💡 이제 로컬에서 이 .pt 파일을 사용하세요!")
    else:
        print("❌ 변환 실패!")

if __name__ == "__main__":
    main()

🔧 Whisper 모델 .pt 변환 도구
🔄 Transformers 모델을 .pt 파일로 변환
📋 필요한 파일들 확인:
   ✅ config.json
   ✅ model.safetensors

🔄 모델 로딩 중...
✅ 모델 로드 완료!
📊 모델 정보:
   - 모델 타입: WhisperForConditionalGeneration
   - 어휘 크기: 51865
   - 모델 크기: 241,734,912 parameters

💾 .pt 파일로 저장 중: whisper_pronunciation_model.pt
✅ 저장 완료!
📊 파일 크기: 926.0 MB

🔍 저장된 파일 검증 중...


/tmp/ipykernel_29713/3837886129.py:86: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_data = torch.load(output_pt_file, map_location='cpu')


✅ 검증 완료! 포함된 데이터:
   - model_state_dict
   - model_config
   - tokenizer
   - feature_extractor
   - processor_config
   - training_info

🧪 변환된 .pt 모델 테스트
🔄 .pt 파일에서 모델 로딩: whisper_pronunciation_model.pt


/tmp/ipykernel_29713/3837886129.py:105: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(pt_file_path, map_location='cpu')


✅ .pt 파일에서 모델 로드 완료!
🔥 GPU로 모델 이동 완료
🔍 더미 데이터로 추론 테스트...
✅ 추론 테스트 성공!
📝 더미 결과:  (static)

🎉 변환 완료!
📁 사용할 파일: whisper_pronunciation_model.pt
💡 이제 로컬에서 이 .pt 파일을 사용하세요!


In [17]:
import os
import torch
import numpy as np
from transformers import WhisperProcessor, WhisperForConditionalGeneration

def load_trained_model(model_path="whisper_pronunciation_finetuned"):
    """저장된 발음 모델 불러오기"""
    print(f"🔄 모델 로딩 중: {model_path}")
    
    if not os.path.exists(model_path):
        print(f"❌ 모델 폴더가 없습니다: {model_path}")
        return None, None
    
    try:
        model = WhisperForConditionalGeneration.from_pretrained(model_path)
        processor = WhisperProcessor.from_pretrained(model_path)
        
        if torch.cuda.is_available():
            model = model.cuda()
            print("🔥 GPU 사용 중")
        else:
            print("💻 CPU 사용 중")
            
        print("✅ 모델 로드 완료!")
        return model, processor
        
    except Exception as e:
        print(f"❌ 모델 로드 실패: {e}")
        return None, None

def load_pcm_file(pcm_file_path, sample_rate=16000):
    """PCM 파일을 로드하여 numpy 배열로 변환"""
    try:
        with open(pcm_file_path, 'rb') as f:
            pcm_data = f.read()
        
        np_pcm = np.frombuffer(pcm_data, dtype=np.int16)
        
        # 사용자 전처리 방식 적용
        normalized = np_pcm / np.max(np.abs(np_pcm))
        
        print(f"   📊 PCM 데이터 길이: {len(normalized)} 샘플")
        print(f"   📊 오디오 길이: {len(normalized)/sample_rate:.2f}초")
        print(f"   📊 오디오 범위: [{normalized.min():.3f}, {normalized.max():.3f}]")
        
        return normalized.astype(np.float32)
        
    except Exception as e:
        print(f"   ❌ PCM 파일 로드 실패: {e}")
        return None

def test_different_decoding_strategies(model, processor, audio):
    """다양한 디코딩 전략 테스트"""
    
    print("🧪 다양한 디코딩 전략 테스트")
    print("=" * 80)
    
    # 입력 특징 추출
    input_features = processor(
        audio, 
        sampling_rate=16000, 
        return_tensors="pt"
    ).input_features
    
    if torch.cuda.is_available():
        input_features = input_features.cuda()
    
    strategies = [
        {
            "name": "1️⃣ 반복 방지 강화",
            "params": {
                "max_length": 200,  # 길이 줄임
                "num_beams": 1,
                "do_sample": False,
                "repetition_penalty": 2.0,  # 🎯 반복 페널티 강화
                "no_repeat_ngram_size": 3,  # 🎯 3-gram 반복 방지
            }
        },
        {
            "name": "2️⃣ 빔 서치 + 반복 방지",
            "params": {
                "max_length": 150,
                "num_beams": 3,  # 🎯 빔 서치 사용
                "do_sample": False,
                "repetition_penalty": 1.5,
                "no_repeat_ngram_size": 2,
                "early_stopping": True,
            }
        },
        {
            "name": "3️⃣ 샘플링 기반",
            "params": {
                "max_length": 150,
                "num_beams": 1,
                "do_sample": True,  # 🎯 샘플링 활성화
                "temperature": 0.7,  # 🎯 창의성 조절
                "top_p": 0.9,       # 🎯 상위 90% 토큰만 사용
                "repetition_penalty": 1.3,
            }
        },
        {
            "name": "4️⃣ 보수적 설정",
            "params": {
                "max_length": 100,  # 🎯 매우 짧게
                "num_beams": 1,
                "do_sample": False,
                "repetition_penalty": 3.0,  # 🎯 매우 강한 반복 페널티
                "no_repeat_ngram_size": 2,
                "pad_token_id": processor.tokenizer.eos_token_id,
            }
        },
        {
            "name": "5️⃣ 길이 제한 + 조기 종료",
            "params": {
                "max_new_tokens": 50,  # 🎯 새 토큰 수 제한
                "num_beams": 2,
                "do_sample": False,
                "repetition_penalty": 2.5,
                "no_repeat_ngram_size": 4,
                "early_stopping": True,
                "eos_token_id": processor.tokenizer.eos_token_id,
            }
        }
    ]
    
    results = []
    
    for strategy in strategies:
        print(f"\n{strategy['name']}")
        print("-" * 60)
        
        try:
            with torch.no_grad():
                predicted_ids = model.generate(
                    input_features,
                    **strategy['params']
                )
            
            transcription = processor.tokenizer.batch_decode(
                predicted_ids, 
                skip_special_tokens=True
            )[0]
            
            # 결과 분석
            result_length = len(transcription)
            has_repetition = check_repetition(transcription)
            has_broken_chars = check_broken_chars(transcription)
            
            print(f"📝 결과 (길이: {result_length}):")
            print(f"   {transcription[:200]}{'...' if len(transcription) > 200 else ''}")
            print(f"🔍 분석:")
            print(f"   - 반복 패턴: {'❌ 있음' if has_repetition else '✅ 없음'}")
            print(f"   - 깨진 문자: {'❌ 있음' if has_broken_chars else '✅ 없음'}")
            
            results.append({
                'strategy': strategy['name'],
                'result': transcription,
                'length': result_length,
                'has_repetition': has_repetition,
                'has_broken_chars': has_broken_chars
            })
            
        except Exception as e:
            print(f"❌ 전략 실패: {e}")
            results.append({
                'strategy': strategy['name'],
                'result': None,
                'error': str(e)
            })
    
    return results

def check_repetition(text):
    """반복 패턴 감지"""
    words = text.split()
    if len(words) < 6:
        return False
    
    # 연속된 3개 단어가 반복되는지 확인
    for i in range(len(words) - 5):
        if words[i:i+3] == words[i+3:i+6]:
            return True
    
    return False

def check_broken_chars(text):
    """깨진 문자 감지"""
    broken_patterns = ['�', '맨�', '깨�', '뀨�']
    return any(pattern in text for pattern in broken_patterns)

def find_best_result(results):
    """최적 결과 찾기"""
    
    print("\n🏆 최적 결과 선정")
    print("=" * 80)
    
    valid_results = [r for r in results if r.get('result') and not r.get('error')]
    
    if not valid_results:
        print("❌ 유효한 결과가 없습니다.")
        return None
    
    # 점수 계산 (낮을수록 좋음)
    for result in valid_results:
        score = 0
        
        # 반복 패턴 페널티
        if result['has_repetition']:
            score += 100
            
        # 깨진 문자 페널티  
        if result['has_broken_chars']:
            score += 50
            
        # 너무 길거나 짧으면 페널티
        if result['length'] > 300:
            score += 30
        elif result['length'] < 10:
            score += 20
            
        result['score'] = score
    
    # 점수순 정렬
    valid_results.sort(key=lambda x: x['score'])
    
    print("📊 결과 순위:")
    for i, result in enumerate(valid_results[:3]):
        print(f"   {i+1}. {result['strategy']} (점수: {result['score']})")
        print(f"      📝 {result['result'][:100]}...")
        print()
    
    best = valid_results[0]
    print(f"🥇 최고 결과: {best['strategy']}")
    print(f"📝 전체 결과: {best['result']}")
    
    return best

def main():
    """메인 함수 - 디코딩 파라미터 최적화"""
    
    print("🎯 디코딩 파라미터 최적화 테스트")
    print("=" * 80)
    
    # PCM 파일 경로
    pcm_file = "000025.wav"
    
    # 파일 존재 확인
    if not os.path.exists(pcm_file):
        print(f"❌ PCM 파일이 없습니다: {pcm_file}")
        return
    
    # 모델 로드
    model, processor = load_trained_model()
    if model is None:
        return
    
    # PCM 파일 로드
    print(f"🎵 PCM 파일 처리 중: {pcm_file}")
    audio = load_pcm_file(pcm_file)
    if audio is None:
        return
    
    print()
    
    # 다양한 디코딩 전략 테스트
    results = test_different_decoding_strategies(model, processor, audio)
    
    # 최적 결과 선정
    best_result = find_best_result(results)
    
    if best_result:
        print("\n" + "=" * 80)
        print("🎉 테스트 완료! 최적 디코딩 설정을 찾았습니다.")
        print("💡 이 설정을 실제 추론에서 사용하세요!")

if __name__ == "__main__":
    main()

🎯 디코딩 파라미터 최적화 테스트
🔄 모델 로딩 중: whisper_pronunciation_finetuned
🔥 GPU 사용 중
✅ 모델 로드 완료!
🎵 PCM 파일 처리 중: 000025.wav
   📊 PCM 데이터 길이: 193515 샘플
   📊 오디오 길이: 12.09초
   📊 오디오 범위: [-0.246, 1.000]

🧪 다양한 디코딩 전략 테스트

1️⃣ 반복 방지 강화
------------------------------------------------------------
📝 결과 (길이: 166):
    거기서 아무런 그런 개입이 업써 관찰만 수천 명의 아이드를 쭉 크는 걸 보니까 부모한테 맨처 면허 애착을 하죠. 그래서 그 아들께서 막 끄러지면서 아예 학꽈를 하는 게 아니라, 비행기구가 뭐 이런 거에 대해서 좀 더 조아하는 건지 모르겓떠라고 생닙니다라고 하더라고 해서 그때 한 번쯔메 안 핸는데
🔍 분석:
   - 반복 패턴: ✅ 없음
   - 깨진 문자: ✅ 없음

2️⃣ 빔 서치 + 반복 방지
------------------------------------------------------------
📝 결과 (길이: 60):
    거기서 아무런 개입이 업써 관찰만 수천 명의 아이드를 쭉 크는 걸 보니까 부모한테 맨처 면허 애착을 하죠.
🔍 분석:
   - 반복 패턴: ✅ 없음
   - 깨진 문자: ✅ 없음

3️⃣ 샘플링 기반
------------------------------------------------------------
📝 결과 (길이: 61):
    거기서 아무런 게이블이 업써 관찰만 수천 명의 아이대를 쭉 키는 걸 보니까 부모한테 맨처매네 애착을 하지요?
🔍 분석:
   - 반복 패턴: ✅ 없음
   - 깨진 문자: ✅ 없음

4️⃣ 보수적 설정
------------------------------------------------------------
📝 결과 (길이: 154):
    거기서 아무런 그런

In [20]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformers import WhisperProcessor, WhisperForConditionalGeneration

def load_and_analyze_pcm(pcm_file_path):
    """PCM 파일 로드하고 상세 분석"""
    
    print("🔍 PCM 파일 상세 분석")
    print("=" * 60)
    
    try:
        # PCM 파일 로드
        with open(pcm_file_path, 'rb') as f:
            pcm_data = f.read()
        
        # numpy 배열로 변환
        np_pcm = np.frombuffer(pcm_data, dtype=np.int16)
        
        print(f"📊 원본 PCM 정보:")
        print(f"   - 파일 크기: {len(pcm_data)} bytes")
        print(f"   - 샘플 수: {len(np_pcm)}")
        print(f"   - 길이: {len(np_pcm)/16000:.2f}초")
        print(f"   - 데이터 범위: [{np_pcm.min()}, {np_pcm.max()}]")
        
        # 앞부분 분석 (첫 5초)
        first_5_sec = np_pcm[:16000*5]  # 첫 5초
        print(f"\n📊 첫 5초 분석:")
        print(f"   - 샘플 수: {len(first_5_sec)}")
        print(f"   - 평균 절댓값: {np.mean(np.abs(first_5_sec)):.2f}")
        print(f"   - 최대 절댓값: {np.max(np.abs(first_5_sec))}")
        print(f"   - 0이 아닌 샘플 비율: {np.count_nonzero(first_5_sec)/len(first_5_sec)*100:.1f}%")
        
        # 정규화 (사용자 방식)
        normalized = np_pcm / np.max(np.abs(np_pcm))
        first_5_sec_norm = normalized[:16000*5]
        
        print(f"\n📊 정규화 후 첫 5초:")
        print(f"   - 범위: [{first_5_sec_norm.min():.3f}, {first_5_sec_norm.max():.3f}]")
        print(f"   - 평균 절댓값: {np.mean(np.abs(first_5_sec_norm)):.3f}")
        
        # 시간대별 음성 활동 분석
        print(f"\n📊 시간대별 음성 활동 (1초 단위):")
        for i in range(min(10, int(len(normalized)/16000))):  # 첫 10초
            segment = normalized[i*16000:(i+1)*16000]
            energy = np.mean(np.abs(segment))
            print(f"   {i:2d}초: 에너지={energy:.4f} {'🔊' if energy > 0.01 else '🔇'}")
        
        return normalized, np_pcm
        
    except Exception as e:
        print(f"❌ PCM 분석 실패: {e}")
        return None, None

def test_whisper_input_features(audio_array):
    """Whisper 입력 특성 분석"""
    
    print("\n🔍 Whisper 입력 특성 분석")
    print("=" * 60)
    
    try:
        # 기본 Whisper processor로 특성 추출
        processor = WhisperProcessor.from_pretrained("openai/whisper-small")
        
        # 전체 오디오
        input_features = processor.feature_extractor(
            audio_array,
            sampling_rate=16000,
            return_tensors="pt"
        ).input_features
        
        print(f"📊 입력 특성 정보:")
        print(f"   - 특성 크기: {input_features.shape}")
        print(f"   - 데이터 타입: {input_features.dtype}")
        print(f"   - 값 범위: [{input_features.min():.3f}, {input_features.max():.3f}]")
        
        # 30초씩 분할 처리 (Whisper 기본 방식)
        chunk_size = 30 * 16000  # 30초
        chunks = []
        
        for i in range(0, len(audio_array), chunk_size):
            chunk = audio_array[i:i+chunk_size]
            if len(chunk) < chunk_size:
                # 마지막 청크는 패딩
                padded_chunk = np.zeros(chunk_size, dtype=np.float32)
                padded_chunk[:len(chunk)] = chunk
                chunk = padded_chunk
            
            chunk_features = processor.feature_extractor(
                chunk,
                sampling_rate=16000,
                return_tensors="pt"
            ).input_features
            
            chunks.append(chunk_features)
            
            # 첫 3개 청크만 분석
            if len(chunks) <= 3:
                print(f"   청크 {len(chunks)} ({i//16000}~{(i+chunk_size)//16000}초): {chunk_features.shape}")
        
        return input_features, chunks
        
    except Exception as e:
        print(f"❌ 입력 특성 분석 실패: {e}")
        return None, None

def compare_basic_vs_finetuned(audio_array):
    """기본 Whisper vs 파인튜닝 모델 비교"""
    
    print("\n🔍 기본 Whisper vs 파인튜닝 모델 비교")
    print("=" * 60)
    
    results = {}
    
    # 1. 기본 Whisper 테스트
    try:
        print("🔄 기본 Whisper 테스트...")
        basic_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
        basic_processor = WhisperProcessor.from_pretrained("openai/whisper-small")
        
        if torch.cuda.is_available():
            basic_model = basic_model.cuda()
        
        input_features = basic_processor(
            audio_array,
            sampling_rate=16000,
            return_tensors="pt"
        ).input_features
        
        if torch.cuda.is_available():
            input_features = input_features.cuda()
        
        with torch.no_grad():
            predicted_ids = basic_model.generate(
                input_features,
                language="ko",
                task="transcribe",
                max_length=500  # 길게 설정
            )
        
        basic_result = basic_processor.tokenizer.batch_decode(
            predicted_ids,
            skip_special_tokens=True
        )[0]
        
        results['basic'] = basic_result
        print(f"✅ 기본 Whisper 결과:")
        print(f"   📝 {basic_result}")
        
    except Exception as e:
        print(f"❌ 기본 Whisper 테스트 실패: {e}")
        results['basic'] = None
    
    # 2. 파인튜닝 모델 테스트 (있다면)
    finetuned_path = "whisper_pronunciation_finetuned"
    if os.path.exists(finetuned_path):
        try:
            print(f"\n🔄 파인튜닝 모델 테스트...")
            finetuned_model = WhisperForConditionalGeneration.from_pretrained(finetuned_path)
            finetuned_processor = WhisperProcessor.from_pretrained(finetuned_path)
            
            if torch.cuda.is_available():
                finetuned_model = finetuned_model.cuda()
            
            input_features = finetuned_processor(
                audio_array,
                sampling_rate=16000,
                return_tensors="pt"
            ).input_features
            
            if torch.cuda.is_available():
                input_features = input_features.cuda()
            
            with torch.no_grad():
                predicted_ids = finetuned_model.generate(
                    input_features,
                    max_length=500,
                    repetition_penalty=2.0,
                    no_repeat_ngram_size=3
                )
            
            finetuned_result = finetuned_processor.tokenizer.batch_decode(
                predicted_ids,
                skip_special_tokens=True
            )[0]
            
            results['finetuned'] = finetuned_result
            print(f"✅ 파인튜닝 모델 결과:")
            print(f"   📝 {finetuned_result}")
            
        except Exception as e:
            print(f"❌ 파인튜닝 모델 테스트 실패: {e}")
            results['finetuned'] = None
    else:
        print(f"\n⚠️  파인튜닝 모델 없음: {finetuned_path}")
        results['finetuned'] = None
    
    return results

def analyze_audio_segments(audio_array, segment_length=5):
    """오디오를 구간별로 나누어 분석"""
    
    print(f"\n🔍 {segment_length}초 구간별 분석")
    print("=" * 60)
    
    try:
        basic_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
        basic_processor = WhisperProcessor.from_pretrained("openai/whisper-small")
        
        if torch.cuda.is_available():
            basic_model = basic_model.cuda()
        
        segment_size = segment_length * 16000
        
        for i in range(0, min(len(audio_array), 30*16000), segment_size):  # 최대 30초까지만
            segment = audio_array[i:i+segment_size]
            
            # 너무 짧은 구간은 패딩
            if len(segment) < segment_size:
                padded_segment = np.zeros(segment_size, dtype=np.float32)
                padded_segment[:len(segment)] = segment
                segment = padded_segment
            
            print(f"\n📍 구간 {i//16000}~{(i+segment_size)//16000}초:")
            print(f"   에너지: {np.mean(np.abs(segment)):.4f}")
            
            try:
                input_features = basic_processor(
                    segment,
                    sampling_rate=16000,
                    return_tensors="pt"
                ).input_features
                
                if torch.cuda.is_available():
                    input_features = input_features.cuda()
                
                with torch.no_grad():
                    predicted_ids = basic_model.generate(
                        input_features,
                        language="ko",
                        task="transcribe",
                        max_length=100
                    )
                
                result = basic_processor.tokenizer.batch_decode(
                    predicted_ids,
                    skip_special_tokens=True
                )[0]
                
                print(f"   결과: {result}")
                
            except Exception as e:
                print(f"   ❌ 처리 실패: {e}")
                
    except Exception as e:
        print(f"❌ 구간별 분석 실패: {e}")

def main():
    """메인 디버깅 함수"""
    
    print("🐛 음성 앞부분 누락 문제 디버깅")
    print("=" * 80)
    
    pcm_file = "000025.wav"
    
    if not os.path.exists(pcm_file):
        print(f"❌ PCM 파일이 없습니다: {pcm_file}")
        return
    
    # 1. PCM 파일 상세 분석
    audio_array, raw_pcm = load_and_analyze_pcm(pcm_file)
    
    if audio_array is None:
        return
    
    # 2. Whisper 입력 특성 분석
    input_features, chunks = test_whisper_input_features(audio_array)
    
    # 3. 기본 vs 파인튜닝 모델 비교
    comparison_results = compare_basic_vs_finetuned(audio_array)
    
    # 4. 구간별 분석
    analyze_audio_segments(audio_array, segment_length=5)
    
    # 5. 결과 요약
    print("\n" + "=" * 80)
    print("📋 디버깅 결과 요약")
    print("=" * 80)
    
    print("🔍 확인해야 할 포인트:")
    print("1. 첫 5초 에너지가 충분한가?")
    print("2. 기본 Whisper도 앞부분을 놓치는가?")
    print("3. 구간별 분석에서 첫 구간 결과는?")
    print("4. 파인튜닝 모델과 기본 모델의 차이는?")
    
    if comparison_results.get('basic'):
        basic_starts_with_target = "아이들" in comparison_results['basic'][:20]
        print(f"\n💡 기본 Whisper 첫 부분 인식: {'✅' if basic_starts_with_target else '❌'}")
    
    if comparison_results.get('finetuned'):
        finetuned_starts_with_target = "아이들" in comparison_results['finetuned'][:20]
        print(f"💡 파인튜닝 모델 첫 부분 인식: {'✅' if finetuned_starts_with_target else '❌'}")

if __name__ == "__main__":
    main()

🐛 음성 앞부분 누락 문제 디버깅
🔍 PCM 파일 상세 분석
📊 원본 PCM 정보:
   - 파일 크기: 387030 bytes
   - 샘플 수: 193515
   - 길이: 12.09초
   - 데이터 범위: [-7886, 32000]

📊 첫 5초 분석:
   - 샘플 수: 80000
   - 평균 절댓값: 1062.83
   - 최대 절댓값: 32000
   - 0이 아닌 샘플 비율: 97.9%

📊 정규화 후 첫 5초:
   - 범위: [-0.246, 1.000]
   - 평균 절댓값: 0.033

📊 시간대별 음성 활동 (1초 단위):
    0초: 에너지=0.0355 🔊
    1초: 에너지=0.0400 🔊
    2초: 에너지=0.0283 🔊
    3초: 에너지=0.0299 🔊
    4초: 에너지=0.0325 🔊
    5초: 에너지=0.0355 🔊
    6초: 에너지=0.0318 🔊
    7초: 에너지=0.0284 🔊
    8초: 에너지=0.0326 🔊
    9초: 에너지=0.0256 🔊

🔍 Whisper 입력 특성 분석
📊 입력 특성 정보:
   - 특성 크기: torch.Size([1, 80, 3000])
   - 데이터 타입: torch.float32
   - 값 범위: [-0.860, 1.140]
   청크 1 (0~30초): torch.Size([1, 80, 3000])

🔍 기본 Whisper vs 파인튜닝 모델 비교
🔄 기본 Whisper 테스트...
✅ 기본 Whisper 결과:
   📝  아이들 크는 걸 이렇게 계절 연구소라고 또 있거든요. 미국에. 거기서 아무런 그런 개입을 없이 그냥 쭉 관찰만 수천 명의 아이들을 쭉 크는 걸 보니까 부모한테 맨 처음에는 애착을 하죠.

🔄 파인튜닝 모델 테스트...
✅ 파인튜닝 모델 결과:
   📝  거기서 아무런 그런 개입이 업써 관찰만 수천 명의 아이드를 쭉 크는 걸 보니까 부모한테 맨처 면허 애착을 하죠. 그래서 그 아들께서 막 끄러지면서 아예 학꽈를 하는 게 아니라, 비행

In [23]:
import os
import torch
import numpy as np
from transformers import WhisperProcessor, WhisperForConditionalGeneration

def load_finetuned_model():
    """파인튜닝된 모델 로드"""
    model_path = "whisper_pronunciation_finetuned"
    
    if not os.path.exists(model_path):
        print(f"❌ 모델 폴더가 없습니다: {model_path}")
        return None, None
    
    try:
        model = WhisperForConditionalGeneration.from_pretrained(model_path)
        processor = WhisperProcessor.from_pretrained(model_path)
        
        if torch.cuda.is_available():
            model = model.cuda()
            print("🔥 GPU 사용 중")
        
        print("✅ 파인튜닝 모델 로드 완료!")
        return model, processor
        
    except Exception as e:
        print(f"❌ 모델 로드 실패: {e}")
        return None, None

def load_pcm_with_preprocessing(pcm_file_path, add_silence=True, amplify_start=True):
    """PCM 파일 로드 + 전처리 옵션"""
    
    try:
        with open(pcm_file_path, 'rb') as f:
            pcm_data = f.read()
        
        np_pcm = np.frombuffer(pcm_data, dtype=np.int16)
        normalized = np_pcm / np.max(np.abs(np_pcm))
        
        # 옵션 1: 앞부분에 짧은 무음 추가
        if add_silence:
            silence_duration = 0.1  # 0.1초
            silence_samples = int(16000 * silence_duration)
            silence = np.zeros(silence_samples, dtype=np.float32)
            normalized = np.concatenate([silence, normalized])
            print(f"   🔇 앞부분에 {silence_duration}초 무음 추가")
        
        # 옵션 2: 앞부분 볼륨 증폭
        if amplify_start:
            start_duration = 3.0  # 첫 3초
            start_samples = int(16000 * start_duration)
            if len(normalized) > start_samples:
                amplification = 1.5  # 1.5배 증폭
                normalized[:start_samples] *= amplification
                # 클리핑 방지
                normalized = np.clip(normalized, -1.0, 1.0)
                print(f"   🔊 앞부분 {start_duration}초 볼륨 {amplification}배 증폭")
        
        return normalized.astype(np.float32)
        
    except Exception as e:
        print(f"❌ PCM 로드 실패: {e}")
        return None

def test_different_decoding_approaches(model, processor, audio):
    """다양한 디코딩 접근법 테스트"""
    
    print("🧪 앞부분 누락 해결을 위한 디코딩 테스트")
    print("=" * 80)
    
    input_features = processor(
        audio, 
        sampling_rate=16000, 
        return_tensors="pt"
    ).input_features
    
    if torch.cuda.is_available():
        input_features = input_features.cuda()
    
    approaches = [
        {
            "name": "1️⃣ 언어/태스크 강제 지정",
            "params": {
                "max_length": 500,
                "num_beams": 1,
                "do_sample": False,
                "repetition_penalty": 2.0,
                "no_repeat_ngram_size": 3,
                "language": "ko",
                "task": "transcribe"
            },
            "use_forced_ids": True
        },
        {
            "name": "2️⃣ 시작 토큰 강제 지정",
            "params": {
                "max_length": 500,
                "num_beams": 1,
                "do_sample": False,
                "repetition_penalty": 2.0,
                "no_repeat_ngram_size": 3,
                "decoder_start_token_id": processor.tokenizer.convert_tokens_to_ids("<|startoftranscript|>")
            },
            "use_forced_ids": False
        },
        {
            "name": "3️⃣ 프롬프트 기반 디코딩",
            "params": {
                "max_length": 500,
                "num_beams": 2,
                "do_sample": False,
                "repetition_penalty": 2.0,
                "no_repeat_ngram_size": 3,
            },
            "use_prompt": True,
            "prompt_text": "아이들"
        },
        {
            "name": "4️⃣ 빔 서치 + 길이 페널티",
            "params": {
                "max_length": 500,
                "num_beams": 3,
                "do_sample": False,
                "repetition_penalty": 1.8,
                "no_repeat_ngram_size": 3,
                "length_penalty": 0.8,  # 짧은 결과 선호도 낮춤
                "early_stopping": False
            },
            "use_forced_ids": False
        },
        {
            "name": "5️⃣ 온도 샘플링",
            "params": {
                "max_length": 500,
                "num_beams": 1,
                "do_sample": True,
                "temperature": 0.3,  # 낮은 온도로 보수적 생성
                "repetition_penalty": 2.0,
                "no_repeat_ngram_size": 3,
            },
            "use_forced_ids": False
        }
    ]
    
    results = []
    
    for approach in approaches:
        print(f"\n{approach['name']}")
        print("-" * 60)
        
        try:
            params = approach['params'].copy()
            
            # 강제 디코더 ID 사용
            if approach.get('use_forced_ids'):
                forced_decoder_ids = processor.get_decoder_prompt_ids(
                    language="ko", 
                    task="transcribe"
                )
                params['forced_decoder_ids'] = forced_decoder_ids
            
            # 프롬프트 기반 디코딩
            elif approach.get('use_prompt'):
                prompt_text = approach['prompt_text']
                prompt_ids = processor.tokenizer.encode(prompt_text, return_tensors="pt")
                if torch.cuda.is_available():
                    prompt_ids = prompt_ids.cuda()
                params['decoder_input_ids'] = prompt_ids
            
            with torch.no_grad():
                predicted_ids = model.generate(
                    input_features,
                    **params
                )
            
            transcription = processor.tokenizer.batch_decode(
                predicted_ids, 
                skip_special_tokens=True
            )[0]
            
            # 앞부분 인식 여부 확인
            has_beginning = any(word in transcription[:50] for word in ["아이들", "크는", "계절"])
            
            print(f"📝 결과: {transcription}")
            print(f"🔍 앞부분 인식: {'✅' if has_beginning else '❌'}")
            
            results.append({
                'approach': approach['name'],
                'result': transcription,
                'has_beginning': has_beginning,
                'length': len(transcription)
            })
            
        except Exception as e:
            print(f"❌ 실패: {e}")
            results.append({
                'approach': approach['name'],
                'result': None,
                'error': str(e)
            })
    
    return results

def test_chunked_processing(model, processor, audio):
    """청크 단위 처리로 앞부분 강제 인식"""
    
    print("\n🔄 청크 단위 처리 테스트")
    print("=" * 60)
    
    try:
        # 첫 10초만 따로 처리
        chunk_duration = 10  # 초
        chunk_samples = chunk_duration * 16000
        
        if len(audio) > chunk_samples:
            first_chunk = audio[:chunk_samples]
            remaining_chunk = audio[chunk_samples:]
        else:
            first_chunk = audio
            remaining_chunk = None
        
        print(f"📊 첫 청크: {len(first_chunk)/16000:.1f}초")
        
        # 첫 청크 처리
        input_features = processor(
            first_chunk,
            sampling_rate=16000,
            return_tensors="pt"
        ).input_features
        
        if torch.cuda.is_available():
            input_features = input_features.cuda()
        
        with torch.no_grad():
            predicted_ids = model.generate(
                input_features,
                max_length=300,
                num_beams=2,
                repetition_penalty=2.0,
                no_repeat_ngram_size=3,
                language="ko",
                task="transcribe"
            )
        
        first_result = processor.tokenizer.batch_decode(
            predicted_ids,
            skip_special_tokens=True
        )[0]
        
        print(f"📝 첫 청크 결과: {first_result}")
        
        # 나머지 청크도 처리 (있다면)
        full_result = first_result
        
        if remaining_chunk is not None and len(remaining_chunk) > 16000:  # 1초 이상
            print(f"📊 나머지 청크: {len(remaining_chunk)/16000:.1f}초")
            
            input_features = processor(
                remaining_chunk,
                sampling_rate=16000,
                return_tensors="pt"
            ).input_features
            
            if torch.cuda.is_available():
                input_features = input_features.cuda()
            
            with torch.no_grad():
                predicted_ids = model.generate(
                    input_features,
                    max_length=300,
                    num_beams=2,
                    repetition_penalty=2.0,
                    no_repeat_ngram_size=3
                )
            
            remaining_result = processor.tokenizer.batch_decode(
                predicted_ids,
                skip_special_tokens=True
            )[0]
            
            print(f"📝 나머지 청크 결과: {remaining_result}")
            full_result = first_result + " " + remaining_result
        
        print(f"📝 전체 결과: {full_result}")
        return full_result
        
    except Exception as e:
        print(f"❌ 청크 처리 실패: {e}")
        return None

def main():
    """메인 함수"""
    
    print("🔧 파인튜닝 모델 앞부분 누락 해결")
    print("=" * 80)
    
    # 모델 로드
    model, processor = load_finetuned_model()
    if model is None:
        return
    
    # PCM 파일 로드 (전처리 옵션 적용)
    pcm_file = "000025.wav"
    
    print("\n🎵 PCM 파일 로드 중...")
    audio = load_pcm_with_preprocessing(
        pcm_file, 
        add_silence=True,    # 앞부분 무음 추가
        amplify_start=True   # 앞부분 볼륨 증폭
    )
    
    if audio is None:
        return
    
    print(f"   📊 전처리 후 길이: {len(audio)/16000:.2f}초")
    
    # 다양한 디코딩 접근법 테스트
    results = test_different_decoding_approaches(model, processor, audio)
    
    # 청크 단위 처리 테스트
    chunked_result = test_chunked_processing(model, processor, audio)
    
    # 결과 요약
    print("\n" + "=" * 80)
    print("📊 해결 방법 평가")
    print("=" * 80)
    
    successful_approaches = [r for r in results if r.get('has_beginning')]
    
    if successful_approaches:
        print("✅ 앞부분 인식 성공한 방법들:")
        for approach in successful_approaches:
            print(f"   - {approach['approach']}")
            print(f"     결과: {approach['result'][:100]}...")
    else:
        print("❌ 모든 방법이 앞부분 인식 실패")
        print("💡 추천: 청크 단위 처리나 모델 재학습 고려")
    
    if chunked_result and "아이들" in chunked_result[:50]:
        print("\n✅ 청크 단위 처리로 앞부분 인식 성공!")

if __name__ == "__main__":
    main()

🔧 파인튜닝 모델 앞부분 누락 해결
🔥 GPU 사용 중
✅ 파인튜닝 모델 로드 완료!

🎵 PCM 파일 로드 중...
   🔇 앞부분에 0.1초 무음 추가
   🔊 앞부분 3.0초 볼륨 1.5배 증폭
   📊 전처리 후 길이: 12.19초
🧪 앞부분 누락 해결을 위한 디코딩 테스트

1️⃣ 언어/태스크 강제 지정
------------------------------------------------------------
📝 결과:  아이들 크는 거를 계젤 연구소라고 또 있거든요. 거기서 아무런 개입이 업쌔 그냥 쭉 관찰만 수천 명의 아이드를 주는 걸 보니까 부모한테 맨처 면허에 착을 하죠?
🔍 앞부분 인식: ✅

2️⃣ 시작 토큰 강제 지정
------------------------------------------------------------
📝 결과:  아이들 크는 거를 계젤 연구소라고 또 있거든요. 거기서 아무런 개입이 업쌔 그냥 쭉 관찰만 수천 명의 아이드를 주는 걸 보니까 부모한테 맨처 면허에 착을 하죠?
🔍 앞부분 인식: ✅

3️⃣ 프롬프트 기반 디코딩
------------------------------------------------------------
📝 결과: 아이들는 거를 계젤 연구소라고 또 있거든요 미국에 거기서 아무런 개입이 업써 관찰만 수천 명의 아이드를 쭉 크는 걸 보니까 부모한테 맨처 면허 애착을 하죠
🔍 앞부분 인식: ✅

4️⃣ 빔 서치 + 길이 페널티
------------------------------------------------------------
📝 결과:  아이들 크는 거를 이렇게 계젤 연구소라고 또 있거든요 미국에 거기서 아무런 그런 뭐 개입이 업씨 그냥 쭉 관찰만 수천 명의 아이드를 쭉 키는 걸 보니까 부모한테 맨처 면허 애착을 하죠.
🔍 앞부분 인식: ✅

5️⃣ 온도 샘플링
------------------------------------------------------------
📝 결과:  아